# Project Overview

> This is an in-depth notebook which explores the Austin Housing Dataset through several different models. The notebook includes a thorough EDA and cleaning section, natural language processing on text descriptions, exploration of different models using different categorical methods (one-hot encoding vs target encoding) with extensive parameter tuning, an evaluation of the final model, and visualizations.

* **Business Objective**


* **Notebook Preparation**
    * Importing our Modules


* **Preprocessing**
    * EDA and Cleaning
        * Scaling target value for Time Series
        * Duplicates
        * Outlier Detection
        * Missing Data
        * Binary Data
        * Studying our Target Variable
    * Natural Language Processing
    * Create Holdout Set
    * Feature Engineering
    * Correlations and Multicollinearity
    * EDA & Process Train Set
        * Categoricals
        * Continuous
            * Standardize Continuous Data
            * Find Interactions
            * Adding Polynomial Features
        * NLP
    * Process Test Set
        * Categoricals
        * Continuous
        * NLP
    * Create Train/Test Final


* **Model Explorations**
    * Picking our Base Features
    * Linear Regressions
        * Basic LR with Top Features One-Hot Encoded
        * Basic LR with Top Features Target Encoded
        * LR with ALL model features
        * Linear Regression with various Feature Selection Methods
            * Permutation Importance
            * Forward-Backward Selector
            * RFECV
    * K-Nearest Neighbors
    * Support Vector Regression
    * Decision Tree Models
        * Decision Tree - One Hot Encoded
            * Random Forest Regressor
            * XGBoost
        * Decision Tree - Target Encoded
            * Random Forest Regressor
            * XGBoost



* **Regression Results and Model Selection**
    * Evaluate results of all attempted models and choose best model


* **Final Model**
    * Process and train final model on data
    * Saving model assets for later use
    * Evaluating final model stats


* Making Predictions with New Data
    * Making new predictions on one entry and imported csv


* Visualizations
    * Feature visualizations


> Analysis
* Statistical analysis and explanations

> Conclusions and Recommendations
* Answers to business questions

> Future Work

> Explanation of Attempts - Feature Engineering/Selection


# Objective

Build a model that accurately predicts house prices in Austin

# Notebook Preparation

In [2]:
# data processing tools
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from math import sqrt
import itertools
from collections import Counter

# model tools
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import neighbors
import xgboost as xgb

# NLP tools
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Visualization tools
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Preprocessing

In [3]:
df = pd.read_pickle("listings_cleaned_na.pkl")
df

address/community address/neighborhood address/streetAddress  \
0                    NaN                  NaN   60 Terrace View Ave   
1                    NaN                  NaN        625 W 246th St   
2                    NaN                  NaN        716 W 231st St   
3                    NaN                  NaN        750 W 232nd St   
4                    NaN                  NaN     24 Cooper St #5CD   
...                  ...                  ...                   ...   
33839                NaN                  NaN        93-19 71st Ave   
33840                NaN                  NaN         6829 Manse St   
33841                NaN                  NaN       82 Greenway Ter   
33842                NaN                  NaN       86 Greenway Ter   
33843                NaN                  NaN         7049 Manse St   

      address/subdivision  address/zipcode  bathrooms  bedrooms    dateposted  \
0                     NaN          10463.0        2.0       5.0  1.610134e+12   
1                     NaN          10471.0        8.0       8.0  1.595968e+12   
2                     NaN          10463.0        3.0       4.0  1.592668e+12   
3                     NaN          10463.0        6.0       5.0  1.600814e+12   
4                     NaN          10034.0        2.0       3.0  1.611091e+12   
...                   ...              ...        ...       ...           ...   
33839                 NaN          11375.0        2.0       4.0           NaN   
33840                 NaN          11375.0        2.0       3.0           NaN   
33841                 NaN          11375.0        6.0       6.0           NaN   
33842                 NaN          11375.0        5.0       6.0           NaN   
33843                 NaN          11375.0        2.0       3.0           NaN   

                                             description homeStatus  \
0      Discover Marble Hill, a neighborhood rich with...   FOR_SALE   
1      EXCLUSIVE BRAND NEW\nLavish Newly Built 8-Bd. ...   FOR_SALE   
2      This 4233 square foot single family home has 4...   FOR_SALE   
3      EXCLUSIVE NEW TO MARKET\nPrime Renovation Oppo...   FOR_SALE   
4      Due to Coronavirus 19, outbreak, ALL showings ...   FOR_SALE   
...                                                  ...        ...   
33839  This house is a rare combination of superb loc...       SOLD   
33840  Wonderful 1 Family Home. First Floor Features ...       SOLD   
33841  "DISTINQUISHED FIELDSTONE TOWNHOUSE TREASURE"\...       SOLD   
33842  EXCLUSIVE LISTING OF TERRACE SOTHEBY'S INTERNA...       SOLD   
33843  7049 Manse St, Flushing, NY 11375 is a single ...       SOLD   

        latitude  livingArea  longitude      price  priceHistory  \
0      40.877743      1889.0 -73.910866   799999.0           NaN   
1      40.892689      7000.0 -73.910667  3995000.0           NaN   
2      40.883419      4233.0 -73.918106  1495000.0           NaN   
3      40.885033      7000.0 -73.917793  3450000.0           NaN   
4      40.867687       994.0 -73.924606   230000.0           NaN   
...          ...         ...        ...        ...           ...   
33839  40.712009      2200.0 -73.850281  1255000.0           NaN   
33840  40.714203      2417.0 -73.855263   825000.0           NaN   
33841  40.717163      6085.0 -73.843124  2704000.0           NaN   
33842  40.717052      4564.0 -73.843025  2750000.0           NaN   
33843  40.712387      1450.0 -73.850288   820000.0           NaN   

       priceHistory/0/buyerAgent          priceHistory/0/buyerAgent/name  \
0                            NaN                                     NaN   
1                            NaN                                     NaN   
2                            NaN                                     NaN   
3                            NaN                                     NaN   
4                            NaN                                     NaN   
...                          ...                       

## EDA and Cleaning

### Scale Time Series

We need to bring all of our home sale prices to the same time scale. It's easy to ignore that these homes were sold over the space of many years, but a year is a long time for real estate. We'll need to appreciate all sale prices into our most recent time series, using months.

In [ ]:
df.rename(columns={'latestPrice':'price'}, inplace=True)

In [ ]:
df['latest_saleyear'].unique()

# our data spans all the way from 2019 to 2021.

In [ ]:
df['latest_saledate'].min()
# earliest sale in january 2018

In [ ]:
df['latest_saledate'].max()
# last sale date january 2021

We need to look to some external sources to get house price appreciation info. I went to the Austin Board of Realtors site to get this info from Texas A&M for the Austin metro area that this data covers  
https://www.recenter.tamu.edu/data/housing-activity/#!/activity/MSA/Austin-Round_Rock

* Jan 2018 median: 287000
* Jan 2019 median: 294000 +2.4%
* Jan 2020 median: 305000 +3.7%
* Jan 2021 median: 363830 +19.3% (!!!)

In [ ]:
jan2018_to_2019 = .024/12
jan2019_to_2020 = .037/12
jan2020_to_2021 = .193/12

In [ ]:
for i in range(1,13):
    df['time_series'] = df['latest_salemonth'].apply(lambda x: abs(x-13))

In [ ]:
# adjust 2018 to 2019
df.loc[df['latest_saleyear']==2018, 'adj_price'] = df.loc[df['latest_saleyear']==2018].apply(lambda x: int( (x['price'])*(1+(jan2018_to_2019*x['time_series']))*(1+(jan2019_to_2020*x['time_series']))*(1+(jan2020_to_2021*x['time_series'])) ) , axis=1 )

# adjust 2019 to 2020                                                            
df.loc[df['latest_saleyear']==2019, 'adj_price'] = df.loc[df['latest_saleyear']==2019].apply(lambda x: int( (x['price'])*(1+(jan2019_to_2020*x['time_series']))*(1+(jan2020_to_2021*x['time_series'])) ) , axis=1)
                                                                                  
# adjust 2020 to 2021
df.loc[df['latest_saleyear']==2020, 'adj_price'] = df.loc[df['latest_saleyear']==2020].apply(lambda x: int( (x['price'])*(1+(jan2020_to_2021*x['time_series'])) ), axis=1)

#copy 2021 to self
df.loc[df['latest_saleyear']==2021, 'adj_price'] = df.loc[df['latest_saleyear']==2021].apply(lambda x: int(x['price']), axis=1)

In [ ]:
# rename original price column and make adj_price our price column
df.rename(columns={'price' : 'orig_price', 'adj_price' : 'price'}, inplace=True)

# put price at the front of the data frame
df.set_index('price', inplace=True)
df.reset_index(inplace=True)

### Checking homeType

In [ ]:
# what are the homeTypes?

df['homeType'].value_counts(normalize=True)

In [ ]:
# Ultimately, with Single Family, Condo and Townhouse making up most of the data, we are going to remove all multi-family type listings
df = df.loc[((df['homeType'] == 'Single Family') | (df['homeType'] == 'Condo')) | (df['homeType'] == 'Townhouse')]

### Duplicate Data

In [ ]:
# check for duplicate data

df[df.duplicated(subset=['latitude','longitude'], keep=False)].sort_values('latitude')

# no duplicate data

### Outlier Detection

In [ ]:
'''# plotting latitude and longitude as a visual scatter plot to look for location-based outliers

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="longitude", y="latitude", hue="price", palette="magma_r");'''

Using latitude and longitude, we make a visual map of the Austin area that lets us see any map outliers. There don't appear to be any zones that are well outside of the Austin area, except for just a few down in the lower SE area. So we might plan to cut off our latitude just above 30.1.

This visualization suggests that location is very important to home price. We'll check that out more directly.

In [ ]:
# drop latitidue below 30.12 to remove the few outliers in the SE
df.drop(df[df['latitude']<30.12].index , inplace=True)

In [ ]:
# looking for outliers in the percentiles

df.describe()

We see potential outliers in price, lotSizeSqFt, livingAreaSqFt, and numOfBathrooms, numOfBedrooms, propertyTaxRate, garageSpaces, parkingSpaces.

In [ ]:
'''# check how our histograms are looking
df.hist(figsize=(18,15), bins=100);
'''

In [ ]:
#check what is going on with the lotSizeSqFt outliers by sorting descending
df.sort_values('lotSizeSqFt', ascending=False).head(5)

# This top listing is legitimate. But we have a problem here where condo and townhouse listings are using the 
# size of the overall lot for their lot, and that isn't really accurate/representative
# We'll fix this in a little bit

In [ ]:
#check what is going on with the livingAreaSqFt outliers by sorting ascending
df.sort_values('livingAreaSqFt', ascending=True).head(5)

# just tiny houses I guess ?

In [ ]:
#check what is going on with the livingAreaSqFt outliers by sorting descending
df.sort_values('livingAreaSqFt', ascending=False).head(5)

In [ ]:
# we're dropping the top two listings here. One is a lot, and the other is clearly mistaken.
df.drop(index=[705, 2557], inplace=True)

In [ ]:
#check what is going on with the numOfBathrooms outliers by sorting descending
df.sort_values('numOfBathrooms', ascending=False).head(5)

In [ ]:
# I'm going to say this top listing has 2.5 bathrooms not 27. That is clearly a typo.
df.loc[df.index==2838, 'numOfBathrooms'] = 2.5

In [ ]:
#check what is going on with the numOfBathrooms outliers by sorting ascending
df.sort_values('numOfBathrooms', ascending=True).head(5)

In [ ]:
# most listings with 0 bathrooms also have 0 bedrooms. This is clearly wrong, but I'm not going to guess if there are no bedrooms.
# I will impute typical bathroom count per bedroom count based on year built
# then drop any remaining listings with 0 bathrooms and 0 bedrooms

df.loc[(df['numOfBathrooms']==0) & (df['numOfBedrooms']>0) & (df['yearBuilt'] > 1989), 'numOfBathrooms'] = 2
df.loc[(df['numOfBathrooms']==0) & (df['numOfBedrooms']>0) & (df['yearBuilt'] <= 1989), 'numOfBathrooms'] = 1
df.loc[(df['numOfBathrooms']==0) & (df['numOfBedrooms']>=3) & (df['yearBuilt'] > 1989), 'numOfBathrooms'] = 2.5
df.loc[(df['numOfBathrooms']==0) & (df['numOfBedrooms']>=3) & (df['yearBuilt'] <= 1989), 'numOfBathrooms'] = 2

df.drop(df[df['numOfBathrooms']==0].index, inplace=True)
df.drop(df[df['numOfBedrooms']==0].index, inplace=True)

In [ ]:
#check what is going on with the numOfBedrooms outliers by sorting descending
df.sort_values('numOfBedrooms', ascending=False).head(5)

In [ ]:
# That condo is supposed to have 2 bathrooms, not 20.
df.loc[df.index==8597, 'numOfBedrooms'] = 2


In [ ]:
#check what is going on with the propertyTaxRate outliers by sorting descending
df.sort_values('propertyTaxRate', ascending=False).head(5)

# looks like some of these are just in a higher tax rate area.

In [ ]:
#check what is going on with the garageSpaces outliers by sorting descending
df.sort_values('garageSpaces', ascending=False).head(10)

In [ ]:
# a bunch of these garage spaces are definitely just bogus numbers. I'm going to force change a lot of them to numbers that make sense
df.loc[(df['garageSpaces'] > 3) & (df['price'] < 1000000) & (df['homeType'] == 'Single Family'), 'garageSpaces'] = 3
df.loc[(df['garageSpaces'] > 5) & (df['price'] > 1000000)& (df['homeType'] == 'Single Family'), 'garageSpaces'] = 4
df.loc[df.index==6885, 'garageSpaces'] = 2

In [ ]:
#check what is going on with the parkingSpaces outliers by sorting descending
df.sort_values('parkingSpaces', ascending=False).head(5)

In [ ]:
# We are going to do the same forced conversions on parking spaces
df.loc[(df['parkingSpaces'] > 3) & (df['price'] < 1000000 & (df['homeType'] == 'Single Family')), 'parkingSpaces'] = 3
df.loc[(df['parkingSpaces'] > 5) & (df['price'] > 1000000& (df['homeType'] == 'Single Family')), 'parkingSpaces'] = 5
df.loc[df.index==6885, 'parkingSpaces'] = 2

df.sort_values('parkingSpaces', ascending=False).head(5)

In [ ]:
df['city'].value_counts()

In [ ]:
'''# check how our histograms are looking for our columns that seem to have outliers

df.hist(figsize=(18,15), bins=100);'''

For the square footage variables, I ultimately concluded that extremely large houses and lots are so seriously under-represented in the dataset that we won't be able to reliably predict on them anyway and they are better left off.

Ultimately I opt to remove via IQR on these items. 

In order to prevent a lot of data loss in this way, I kept IQR range of 1.6 instead of the standard 1.5

In [ ]:
# A lot of our variables are not normally shaped, so we can't reliably remove outliers via standard deviation.
# We will use IQR to remove our outliers with the following function

def iqr_outliers(column):
    """return the lower range and upper range for the data based on IQR*1.6"""
    Q1,Q3 = np.percentile(column , [25,75])
    iqr = Q3 - Q1
    lower_range = Q1 - (1.6 * iqr)
    upper_range = Q3 + (1.6 * iqr)
    return lower_range,upper_range  

In [ ]:
# determing our IQR for price, lot size, sq footage and longitude
lotlower,lotupper = iqr_outliers(df.lotSizeSqFt)
sqftlower, sqftupper = iqr_outliers(df.livingAreaSqFt)

# dropping the things outside of our lower and upper range
df.drop(df[ (df.lotSizeSqFt > lotupper) | (df.lotSizeSqFt < lotlower) ].index , inplace=True)
df.drop(df[ (df.livingAreaSqFt > sqftupper) | (df.livingAreaSqFt < sqftlower) ].index , inplace=True)

In [ ]:
# We'll imputing the median lot size into condo and townhouse listings that are over-listed for lot size square feet

df.loc[(df['homeType']=='Condo') | (df['homeType']=='Townhouse') & (df['livingAreaSqFt']<1200) & (df['lotSizeSqFt']>8000), 'lotSizeSqFt'] = df['lotSizeSqFt'].median()

In [ ]:
'''# check how our histograms are looking

df.hist(figsize=(18,20), bins=100);

# much better'''

#### Manually locating price outliers

In [ ]:
# we're using the median house value for a zip code to determine the zip code's sort, so we can visualize the zip code

# group our dataframe by zipcode on median home price, sorted ascending. 
zipsorted = pd.DataFrame(df.groupby('zipcode')['price'].median().sort_values(ascending=True))

# divide our dataframe into groups with entries per group as specified above,
# and assign this number to a new column
zipsorted['rank'] = np.divmod(np.arange(len(zipsorted)), 1)[0]+1

# function that looks up a segment that a data entry belongs to
def make_group(x, frame, column):
    y = frame.loc[(frame.index == x)][column]
    z = np.array(y)
    z[0]
    return z[0]

# make a new column on our dataframe. Look up each zip entry's group, and append to the column.
df['zip_rank'] = df['zipcode'].apply(lambda x: make_group(x, zipsorted, 'rank'))

# apply the median home price per zip code to the data frame
df['median_zip'] = df['zipcode'].apply(lambda x: round(df.loc[df['zipcode']==x]['price'].median(), 0))

In [ ]:
'''# visualize zip code as a color function

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['median_zip'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Median Home Price per Zip', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Zip Code Median, by Zip Code Median Rank', fontsize=20)
;'''

We can see that a few of our zip codes are very high value. There are also some clear outliers in this data set. We'll take care of removing those, and then come back to this visual again later after we've done some cleanup.

In [ ]:
'''# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['livingAreaSqFt'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Feet, by Zip Code Median Rank', fontsize=20)
;'''

In [ ]:
# we're dropping the values above 3 million, and the 3 entries from zipcode 78734
df.drop(df[df['price']>3000000].index, inplace=True)
df.drop(df[df['zipcode']==78734].index, inplace=True)

In [ ]:
# check price stats by zip code and displaying top 30 zip codes by mean

find_zip_outliers = df.groupby('zipcode')['price'].describe()
find_zip_outliers.sort_values('mean', ascending=False).head(35)

# very suspicious values in many zip codes for min

In [ ]:
# anything under 75k is no way a legitimate market value sale. 
# anything in this range is certainly a inter-family sale,
# non-commercial, some weird sale type.
# We are dropping all of those. 
df.drop(df.loc[(df['price'] <= 75000)].index, axis=0, inplace=True)

In [ ]:
# Eliminating outliers on a per-zipcode basis using our IQR 1.6

zipcodes = df['zipcode'].unique()

for i in zipcodes:
    lower, upper = iqr_outliers(df[df['zipcode'] == i]['price'])
    df.drop(df[ ( (df.price > upper) & (df['zipcode'] == i) ) | ( (df.price < lower)  & (df['zipcode'] == i) ) ].index , inplace=True)


In [ ]:
'''#We can check our price per zip code histograms.

df['price'].hist(by=df['zipcode'], figsize=(30,30));    

# some of our zip codes don't have enough sales to give us information'''

In [ ]:
# We're going to drop our few zip codes where we have only a couple of data points

df.drop( df.loc[(df['zipcode']==78653) | (df['zipcode']==78738) | (df['zipcode']==78719)| (df['zipcode']==78652)| (df['zipcode']==78742)].index, axis=0, inplace=True)

In [ ]:
# redo our zip code medians and rankings after outlier removal

# apply the median home price per zip code to the data frame again after outlier removal
df['median_zip'] = df['zipcode'].apply(lambda x: round(df.loc[df['zipcode']==x]['price'].median(), 0))

# group our dataframe by zipcode on median home price, sorted ascending. We want to bin like-medians together.
zipsorted = pd.DataFrame(df.groupby('zipcode')['price'].median().sort_values(ascending=True))

# divide our dataframe into groups with entries per group as specified above,
# and assign this number to a new column
zipsorted['rank'] = np.divmod(np.arange(len(zipsorted)), 1)[0]+1

# make a new column on our dataframe. Look up each zip entry's group, and append to the column.
df['zip_rank'] = df['zipcode'].apply(lambda x: make_group(x, zipsorted, 'rank'))

In [ ]:
'''# re-visualize zip code as a color function, using the median zip after outlier removal. 

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['median_zip'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Zip Code by Median Rank', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Zip Code Median, by Zip Code Median Home Value', fontsize=20);

# save visualization to png
#plt.savefig('images/zip_prices.png')'''

In [ ]:
# apply the median price per square foot per zip code to the data frame
df['pr_sqft'] = df.apply(lambda x: round( (x['price'] / x['livingAreaSqFt'] ), 0), axis=1 )

In [ ]:
'''fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['livingAreaSqFt'], df['pr_sqft'], c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Total Square Footage', fontsize=12)
ax.set_ylabel('Price Per Square Foot', fontsize=12)
ax.set_title('Price Per Square Foot to Total Square Footage, by Zip Code Median Rank', fontsize=20);

# save visualization to png
#plt.savefig('images/zip_prices.png')'''

In [ ]:
#dropping irrationally high pr/sqft
df.drop(df[df['pr_sqft']>1000].index, inplace=True)

In [ ]:
'''# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(df['livingAreaSqFt'], df['price'] /100000, c=df['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Footage, by Zip Code Median Rank', fontsize=20)
;'''

In [ ]:
df['price'].mean()

In [ ]:
low_zips = df.loc[df['median_zip']<500000]
high_zips = df.loc[df['median_zip']>=500000]

In [ ]:
'''# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(low_zips['livingAreaSqFt'], low_zips['price'] /100000, c=low_zips['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Footage, by Zip Code Median Rank \nFor Zip Medians under 500k', fontsize=20)
;'''

In [ ]:
'''# visualize zip code as a color function, on a plot of price per square footage

fig, ax = plt.subplots(figsize=(20, 15))

ax.scatter(high_zips['livingAreaSqFt'], high_zips['price'] /100000, c=high_zips['zip_rank'], cmap='magma_r')

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Price in $100,000', fontsize=12)
ax.set_title('Price per Total Square Footage, by Zip Code Median Rank\nFor Zip Medians over 500k', fontsize=20)
;'''

Here's a fun way to see the improvements to our data quality after we clean outliers! A much deeper color map.

In [ ]:
'''# plotting latitude and longitude as a visual scatter plot. The improved color map actually visually demonstrates
# the removal of extreme price outliers.

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="longitude", y="latitude", hue="price", palette="magma_r");'''

In [ ]:
'''# we can also map our zip codes in this way.

plt.figure(figsize=(25,25))

sns.scatterplot(data=df, x="longitude", y="latitude", hue="zip_rank", palette="magma_r");'''

### Missing Data

In [ ]:
# look for nulls

df.isna().sum()

# no missing data. impressive!

In [ ]:
# check data types

df.dtypes

# data types all look correct

### Binary Data

In [ ]:
# we're going to convert some of our ordinal features to binary 0/1
convert_to_bool = ['numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures', 'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures', 'numOfWaterfrontFeatures', 'numOfWindowFeatures', 'numOfCommunityFeatures']

df_convert_to_bool = df[convert_to_bool]
df_convert_to_bool.describe()

In [ ]:
# Any element that has no features by the 50th percentile is getting converted to a binary 

# change all non-null values > 0 in those columns to 1
df.loc[df['numOfAccessibilityFeatures'] > 0, 'numOfAccessibilityFeatures'] = 1
df.loc[df['numOfPatioAndPorchFeatures'] > 0, 'numOfPatioAndPorchFeatures'] = 1
df.loc[df['numOfSecurityFeatures'] > 0, 'numOfSecurityFeatures'] = 1
df.loc[df['numOfWaterfrontFeatures'] > 0, 'numOfWaterfrontFeatures'] = 1
df.loc[df['numOfWindowFeatures'] > 0, 'numOfWindowFeatures'] = 1
df.loc[df['numOfCommunityFeatures'] > 0, 'numOfCommunityFeatures'] = 1

# now anything that is not a 1 becomes a 0
df.loc[df['numOfAccessibilityFeatures']!= 1, 'numOfAccessibilityFeatures'] = 0
df.loc[df['numOfPatioAndPorchFeatures'] != 1, 'numOfPatioAndPorchFeatures'] = 0
df.loc[df['numOfSecurityFeatures'] != 1, 'numOfSecurityFeatures'] = 0
df.loc[df['numOfWaterfrontFeatures'] != 1, 'numOfWaterfrontFeatures'] = 0
df.loc[df['numOfWindowFeatures'] != 1, 'numOfWindowFeatures'] = 0
df.loc[df['numOfCommunityFeatures'] != 1, 'numOfCommunityFeatures'] = 0

# rename to reflect binary
df.rename(columns={'numOfAccessibilityFeatures' : 'accessibility', 'numOfPatioAndPorchFeatures' : 'patioporch', 'numOfSecurityFeatures': 'security', 
                  'numOfWaterfrontFeatures': 'waterfront', 'numOfWindowFeatures' : 'windowfeatures', 'numOfCommunityFeatures' : 'community'}, inplace=True)

In [ ]:
# convert original boolean columns to binary 0/1
boolean = ['hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView']

for item in boolean:
    df[boolean] = df[boolean].astype(int)

### Study Target Variable

In [ ]:
#histogram and normal probability plot
sns.distplot(df['price'], fit=norm);
fig = plt.figure()

res = stats.probplot(df['price'], plot=plt)

# our sales price histogram is positively skewed and has a high peak
# Our QQ-plot shows that we have heavy tails with right skew

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['price'].skew())
print("Kurtosis: %f" % df['price'].kurt())

# price is highly right skewed
# very positive kurtosis, indicating lots in the tails. We can see those tails in the right skew.

In [ ]:
# log transform our target price to improve normality of distribution
df_target_log = np.log(df['price'])

#histogram and normal probability plot
sns.distplot(df_target_log, fit=norm);
fig = plt.figure()
res = stats.probplot(df_target_log, plot=plt)

# Our target price is more normally distributed when log transformed, so we'll be doing that when we make our model


## Natural Language Processing

Our data set includes the listing text for each sale. We're going to use Natural Language Processing methods to extract relevant information from the listing text to boost the effectiveness of our model.

We're using spaCy and after basic package installation, we also need to load the english language pipeline.

In [ ]:
# Load spaCy with English language processor
nlp = spacy.load("en_core_web_sm")

# add real estate related stop words to default stop word list
nlp.Defaults.stop_words |= {"bedroom", "bathroom","bath","home", "austin", "tx", "pron", "sq", "ft", "rent", "mo",
                            "w", "bed", 'single', 'family', 'contain', 'st', 'dr', 'square', 'foot', 'room', 'square', 'feet',
                            '-pron-', 'garage', 'pflugerville', 'story', '1st', '1story', '2car', '2nd',
                            '2story', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', 'street', 'avenue', 'ave', 
                            'sac', 
                            
                           }

nlp.Defaults.stop_words.remove('is')
nlp.Defaults.stop_words.remove('as')

In [ ]:
# text processing functions for NLP

def preprocessor(word):
    '''processes an individual word to remove punctuation, numbers, special characters etc
    Returns processed word, or blank string if character removal resulted in no word
    ARGUMENT:
    word from line of text'''
    if type(word) == str:
        word = re.sub(r'[^\w\s]', '', word)
        word = re.sub('<[^>]*>', '', word)
        word = re.sub('<[0-9]*>', '', word)
        word = re.sub('[\W]+', '', word.lower())
        try:
            word = int(word)
            word = ''
            return word
        except:
            return word

def word_processor(line):
    '''Takes a line of text. Tokenizes each word of sentence. 
    If token is stop word, goes to next token. If not stop word,
    calls preprocessor on word
    Returns processed words from line
    ARGUMENT: 
    line of text'''
    
    tokens = nlp(line) # nlp reads line and creates tokens from each word  
    words = [] # empty list of words for this line
    
    for token in tokens:
        if token.is_stop == False: # only continues if token is not stop word
            token_preprocessed = preprocessor(token.lemma_) # calls preprocessor on word
            if token_preprocessed != '': # only continues if returned word is not empty
                words.append(token_preprocessed) # appends word to list of words
    return(words) # return list of words for this line

def text_block_processor(text):
    '''Takes a block of text. Divides block into sentences with words lemmatized.
    Sends each sentence to word processor. Concatenates all words into one string
    If the string contains "zestimate", returns a DEFAULT listing note
    Otherwise returns string of cleaned and processed words from text block
    ARGUMENTS:
    block of text
    '''
    
    make_sentences = nlp(text)
    
    sentences_lemmata_list = [sentence.lemma_.lower() for sentence in make_sentences.sents]
    
    these_processed_sentences = ''

    
    for item in sentences_lemmata_list:
        words = word_processor(item)
        line = ' '.join(words)
        these_processed_sentences += (' ' + line)
        
    if 'zestimate' in these_processed_sentences:
        return 'DEFAULT'
    else:
        return these_processed_sentences

In [ ]:
# reset indices on original data frame before making a copy
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [ ]:
'''# copy the description column to a new data frame for text processing
listing_text = pd.DataFrame(df['description'])
listing_text

# cleaning all of the text in our listing_text description field and adding it to the listing_text data frame 
sentences = []

listing_text['sentences'] = None

for row in listing_text.index:
    thistext = listing_text['description'][row]
    the_sentences = text_block_processor(thistext)
    listing_text['sentences'][row] = the_sentences
    sentences += the_sentences

# drop the description field and export our listing_text to csv so we don't have to run it again
listing_text.drop('description', axis=1, inplace=True)

listing_text.to_pickle("listing_text.pkl")'''

In [ ]:
listing_desc = pd.read_pickle("listing_text.pkl")
listing_desc

In [ ]:
# append our listing text to our original data frame
df = pd.concat([df, listing_desc], axis=1)

df

## Create Holdout Set

We need to create our holdout data before any further processing.

The reasons for this are:
   * We will standardize our continuous variables, and you should standardize only on your train set and apply that to your test set.
   * We will be feature engineering on our train set, and applying that later to our test set. We cannot have our test set data leak into our engineered features.
   * We'll be doing some natural language processing, fitting on our train set and applying to our test set. We don't want data leakage for the same reasons as above.
    

In [ ]:
# set our random seed for the notebook. We could randomize this each time the notebook is run,
# but ultimately we want all of our train/test splits to use the same data
randomstate = 2

y = pd.DataFrame(df['price'])
x = df.drop('price', axis=1,)

# creating our train/validation sets and our test sets
train_data, holdout, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=randomstate)

# reset indices to prevent any index mismatches
train_data.reset_index(inplace=True)
train_data.drop('index', axis=1, inplace=True)

holdout.reset_index(inplace=True)
holdout.drop('index', axis=1, inplace=True)

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

## Feature Engineering

In [ ]:
# Adding target encoding, which we will opt to try instead of one-hot with a few models

# smooth mean function by MAx Halford at https://maxhalford.github.io/blog/target-encoding/

def calc_smooth_mean(df, by, on, m, target_df):
    '''input a pandas.DataFrame, a categorical column name, the name of the target column, and a weight .'''
    # Compute the global mean
    mean = df[on].mean() 

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])  
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return round(target_df[by].map(smooth), 0) 

num_of_samples = train_data.shape[0]
zip_samples = num_of_samples/train_data['zipcode'].unique().shape[0]
month_samples = num_of_samples/train_data['latest_salemonth'].unique().shape[0]


# create smooth additive encoded variables for zipcode, year built, and monthsold
train_data['zip_smooth'] = calc_smooth_mean(train_data, 'zipcode', 'price', zip_samples, train_data)
train_data['year_smooth'] = calc_smooth_mean(train_data, 'yearBuilt', 'price', 300, train_data)
train_data['month_smooth'] = calc_smooth_mean(train_data, 'latest_salemonth', 'price', month_samples, train_data)

# Create a wider lat and long zone to calculate an area mean
train_data['lat_zone'] = round(train_data['latitude'], 2)
train_data['long_zone'] = round(train_data['longitude'], 2)

lat_samples = num_of_samples/train_data['lat_zone'].unique().shape[0]
long_samples = num_of_samples/train_data['long_zone'].unique().shape[0]

# calculate smooth mean variables for lat and long, then create an interactive variable describing both together
train_data['lat_smooth'] = calc_smooth_mean(train_data, 'lat_zone', 'price', lat_samples, train_data)
train_data['long_smooth'] = calc_smooth_mean(train_data, 'long_zone', 'price', long_samples, train_data)
train_data['lat_long'] = round(np.sqrt(train_data['lat_smooth'] * train_data['long_smooth']), 0)

## Correlations/Multicollinearity

In [ ]:
# look for multicollinearity of features
fig, ax = plt.subplots(figsize=(20, 20))

sns.heatmap(train_data.corr(), center=0,  
           vmin=-1, vmax=1,  square=True)

# title
plt.title('PEARSON CORRELATION MATRIX', fontsize=18)

plt.show()

In [ ]:
train_data.corr()

In [ ]:
#Get our list of highly correlated feature pairs with following steps:

# save correlation matrix as a new data frame
# converts all values to absolute value
# stacks the row:column pairs into a multindex
# reset the index to set the multindex to seperate columns
# sort values. 0 is the column automatically generated by the stacking
df_correlations = train_data.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns in a new column named "pairs"
df_correlations['pairs'] = list(zip(df_correlations.level_0, df_correlations.level_1))

# set index to pairs
df_correlations.set_index(['pairs'], inplace = True)

# rename our results column to correlation
df_correlations.rename(columns={0: "correlation"}, inplace=True)

# Drop 1:1 correlations to get rid of self pairs
df_correlations.drop(df_correlations[df_correlations['correlation'] == 1.000000].index, inplace=True)

# view pairs above 75% correlation and below 90% correlation (engineered features will correlate with each other above 95%)
df_correlations[(df_correlations.correlation>.75) & (df_correlations.correlation<.95)]


In [ ]:
# Check out our variables correlationg with price
df_correlations = train_data.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_correlations.loc[df_correlations['level_0'] == 'price'].sort_values(0, ascending=False)

We'll drop:

* parkingSpaces, hasGarage, numOfParkingFeatures and keep garageSpaces (higher relationship with Price)
* numOfElementarySchools, and keep numOfPrimarySchools (higher relationship with Price)
* MedianStudentsPerTeacher, keeping avgSchoolRating
* numOfBathrooms correlates with square footage, but I'm not dropping either

We can get a sense of the most important features to our price from our correlation table. Zipcode as a plain variable does not correlate, which makes sense, because without some sort of transformation it is an arbitrary unordered number. We can see how transformed as median_zip or zip_rank it becomes the MOST important contributor to price. We can see here that big contributors to price include
    
    * Lat/long in a target encoded form
    * zip code (in some altered form, not as arbitrary number)
    * livingAreaSqFt
    * numBathrooms
    * avgSchoolRating

In [ ]:
train_data['numOfSchools'] = train_data['numOfPrimarySchools'] + train_data['numOfElementarySchools'] + train_data['numOfMiddleSchools'] + train_data['numOfHighSchools']
holdout['numOfSchools'] = holdout['numOfPrimarySchools']  + holdout['numOfElementarySchools'] + holdout['numOfMiddleSchools'] + holdout['numOfHighSchools']

In [ ]:
# drop multicollinear features
train_data.drop(['parkingSpaces', 'hasGarage', 'numOfElementarySchools', 'numOfPrimarySchools', 'numOfMiddleSchools', 
         'numOfHighSchools', 'MedianStudentsPerTeacher', 'numOfParkingFeatures'], axis=1, inplace=True)
holdout.drop(['parkingSpaces', 'hasGarage', 'numOfElementarySchools', 'numOfPrimarySchools', 'numOfMiddleSchools', 
         'numOfHighSchools', 'MedianStudentsPerTeacher', 'numOfParkingFeatures'], axis=1, inplace=True)

## EDA and Processing Train Set

### Categoricals

In [ ]:
categories = ['zipcode', 'yearBuilt', 'hasAssociation', 'hasCooling', 'hasHeating', 'hasSpa', 'hasView', 'accessibility', 'patioporch', 'security',
          'waterfront', 'windowfeatures', 'community', 'latest_salemonth', 'numOfSchools', 'garageSpaces', 'propertyTaxRate', ]

df_categoricals = train_data[categories]

In [ ]:
# adding price to our dataframe so that we can do some visualizations    

df_categoricals['price'] = train_data['price']

# plot our categoricals as box plots vs price
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
    
f = pd.melt(df_categoricals, id_vars=['price'], value_vars=categories)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")

df_categoricals.drop('price', axis=1, inplace=True)

In [ ]:
# there is only ONE listing with 5 schools, so we will change that one to 4
df_categoricals.loc[df_categoricals['numOfSchools']==5, 'numOfSchools'] = 4

In [ ]:
# binning our year built bins
# We're also saving the bin information to year_bins because we will need the bin information to make new predictions
num_bins = 30
labels = np.array(range(1,num_bins+1))
df_categoricals["year_block"], year_bins = pd.qcut(df_categoricals['yearBuilt'], q=num_bins, retbins=True, labels=labels)

df_categoricals.drop('yearBuilt', axis=1, inplace=True)

# telling Pandas that these columns are categoricals
for item in df_categoricals.columns:
    df_categoricals[item] = df_categoricals[item].astype('category')

In [ ]:
# make a processed bins file for use with linear regression
# We're making TWO categorical sets. One is high one hot encoding. One is low one hot encoding, and the 
# categoricals in that one will be target encoded as continuous instead

high_one_hot_cat =  ['zipcode', 'year_block', 'hasAssociation', 
                 'hasCooling', 'hasHeating', 'hasSpa', 'hasView', 
                 'accessibility', 'patioporch', 'security', 'numOfSchools',
              'waterfront', 'windowfeatures', 'community', 'latest_salemonth',
                    'garageSpaces', 'propertyTaxRate', ]
low_one_hot_cat =  ['hasAssociation', 
                'hasCooling', 'hasHeating', 'hasSpa', 
                'hasView', 'accessibility', 'patioporch', 'numOfSchools',
                'security', 'waterfront', 'windowfeatures', 'community', 'garageSpaces', 'propertyTaxRate', ]

df_cats_high_one_hot = pd.get_dummies(df_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=True)
df_cats_low_one_hot = pd.get_dummies(df_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=True)

### Continuous

In [ ]:
continuous = ['numPriceChanges', 
              'lotSizeSqFt', 'livingAreaSqFt', 'avgSchoolDistance', 
              'avgSchoolRating', 'avgSchoolSize', 'numOfBedrooms', 
              'numOfStories', 'numOfPhotos', 
              'numOfAppliances', 'latest_salemonth',
             'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long'] 

x_continuous = train_data[continuous]
x_continuous['price'] = train_data['price']

Let's look at mean price by month and see if there are any better insights.

In [ ]:
small_cont = ['numPriceChanges', 
              'avgSchoolRating', 'numOfBedrooms', 
              'numOfStories', 'numOfAppliances', 
              'latest_salemonth']
# plot our continuous as box plots vs price
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(x_continuous, id_vars=['price'], value_vars=small_cont)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "price")


In [ ]:
large_cont = ['lotSizeSqFt', 'livingAreaSqFt', 'avgSchoolDistance', 
              'avgSchoolRating', 'avgSchoolSize', 'numOfPhotos',
             'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long']

# check linearity of continuous predictors

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15,25), sharey=True)

for ax, column in zip(axes.flatten(), large_cont):
    ax.scatter(x_continuous[column], x_continuous['price']/100000, label=column, alpha=.1)
    ax.set_title(f'Sale Price vs {column}')
    ax.set_xlabel(column)
    ax.set_ylabel('Sale Price in $100,000')

fig.tight_layout()

Positive relationship observed with:
* lot size
* square footage
* school rating
* number of bedrooms
* lat/long

Negative relationship observed with:
* number of price changes

Others seem neutral/uncertain

In [ ]:
# Checking out our mean sales price for year built scattered versus price shows a polynomial relationship

yearly_prices = train_data.groupby('latest_salemonth')['price'].mean()

plt.scatter(yearly_prices.index, yearly_prices)
plt.title("Linearity check")
plt.xlabel('year built')
plt.ylabel('sales price')
plt.show()

Our average per month looks polynomial.

In [ ]:
# Checking out our mean sales price for latitude  scattered versus price shows a polynomial relationship
lat_prices = train_data.groupby('livingAreaSqFt')['price'].mean()

plt.scatter(lat_prices.index, lat_prices)
plt.title("Linearity check")
plt.xlabel('lat')
plt.ylabel('sales price')
plt.show()

In [ ]:
# Checking out our mean sales price for latitude  scattered versus price shows a polynomial relationship
lat_prices = train_data.groupby('lotSizeSqFt')['price'].mean()

plt.scatter(lat_prices.index, lat_prices)
plt.title("Linearity check")
plt.xlabel('lat')
plt.ylabel('sales price')
plt.show()

In [ ]:
# Checking out our mean sales price for latitude  scattered versus price shows a polynomial relationship
lat_prices = train_data.groupby('avgSchoolSize')['price'].mean()

plt.scatter(lat_prices.index, lat_prices)
plt.title("Linearity check")
plt.xlabel('lat')
plt.ylabel('sales price')
plt.show()

#### Standardize

In [ ]:
# check out our histograms

x_continuous.hist(figsize=(18,15), bins='auto');

In [ ]:
# don't need price in there anymore
x_continuous.drop('price', axis=1, inplace=True)

# standardize all of our values with scikit-learn StandardScaler
scaler = StandardScaler()

#transformed_scaled_continuous = pd.DataFrame(scaler.fit_transform(x_train_cont_log),columns = x_train_cont_log.columns)
scaled_continuous = pd.DataFrame(scaler.fit_transform(x_continuous),columns = x_continuous.columns)
scaled_continuous.head(5)

In [ ]:
# make a processed bins file for use with linear regression
# We're making TWO continuous sets. One is high one hot encoding. One is low one hot encoding, and includes the 
# categoricals that are target encoded as continuous instead

high_one_hot_cont =  ['numPriceChanges', 
              'lotSizeSqFt', 'livingAreaSqFt', 'avgSchoolDistance', 
              'avgSchoolRating', 'avgSchoolSize', 'numOfBedrooms', 
              'numOfStories', 'numOfPhotos', 
              'numOfAppliances']
low_one_hot_cont =  ['numPriceChanges', 
              'lotSizeSqFt', 'livingAreaSqFt', 'avgSchoolDistance', 
              'avgSchoolRating', 'avgSchoolSize', 'numOfBedrooms', 
              'numOfStories', 'numOfPhotos', 
              'numOfAppliances',  
             'zip_smooth', 'year_smooth', 'month_smooth', 'lat_long']

df_cont_high_one_hot = scaled_continuous[high_one_hot_cont]
df_cont_low_one_hot = scaled_continuous[low_one_hot_cont]

#### Finding Interactions

I wrote a function which finds all of the feature combinations possible in our dataset. Then for each combination, the function runs a linear regression with cross validation on 5 folds and gets the r^2 score for the regression including that feature combination. All scores are recorded and r^2 score improvement is assessed, with the resulting table giving the increase in model improvement from a feature combo. 

In [ ]:
'''def test_feature_combinations(price, variables):
    
    """Function takes in target price and a dataframe of independent variables, and 
    tests model improvement for each combination of variables
    ARGUMENTS:
    Y of target values
    X-dataframe of continuous features
    Returns dataframe of score improvements over base score for each interaction combination"""
    
    # select our estimator and our cross validation plan
    regression = LinearRegression()
    cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
    
    # prepare our scoring dataframe
    scoring_df = pd.DataFrame()
    
    # prepare our lists to store our features and scores as we iterate
    scores = []
    feature1 = []
    feature2 = []
    
    # Get a list of all of our features, and remove our target variable 'price' from the list
    features = list(variables.columns)

    # make a list of all of our possible feature combinations
    feature_combos = itertools.combinations(features, 2)
    feature_combos = list(feature_combos)
    
    # set our y-value as our target variable
    y = price
    
    # prepare our x-value with our independent variables. We do an initial split here in order to run a 
    # linear regression to get a base r^2 on our basic model without interactions
    X = variables
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomstate)
    base_score = round(np.mean(cross_val_score(regression, X_train, y_train, scoring='r2', cv=cv)), 4)   
    print("Model base score is ",base_score)
    
    # now we run the regression on each feature combo
    for feature in feature_combos:
        feat1, feat2 = feature[0], feature[1]
        
        # create the test interaction on our data set
        variables['test_interaction'] = variables[feat1] * variables[feat2]
        # create a new X which includes the test interaction and drops our target value
        X = variables
        # make a new split so that our x-splits include the test interaction
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomstate)
        
        # Run a linear regression with cross-val just like our base model, and append the score to our scores list
        new_score = round(np.mean(cross_val_score(regression, X_train, y_train, scoring='r2', cv=cv)), 4)
        scores.append(new_score)
        # put feature 1 on a list
        feature1.append(feat1)
        # put feature 2 on a list
        feature2.append(feat2)
        print(feat1, feat2, new_score)
        
        
    
    # load all of our lists into the scoring dataframe
    scoring_df['feature1'] = feature1
    scoring_df['feature2'] = feature2
    scoring_df['scores'] = scores
    scoring_df['improvement'] = scoring_df['scores'] - base_score
    variables.drop('test_interaction', axis=1, inplace=True)
    
    # return our scoring dataframe to the function
    return scoring_df'''

In [ ]:
# running our function on our continuous variables to look for improvement
# our R2 is much lower for model base score because we aren't including our categorical variables in this improvement assessment

#scoring_df = test_feature_combinations(y_train, df_cont_low_one_hot)

In [ ]:
# showing our improvement scores for our interactions

#scoring_df.sort_values('improvement', ascending=False)

We won't add any interactions. None of these improvements were significant enough.

#### Add polynomial features

In [ ]:
def plot_polys(y, xlabel, title):
    '''Takes in a y-axis, x-axis label, and title and plots with various polynomial levels
    ARGUMENTS:
    y axis variable values
    x-axis label
    visualization title'''
    x = y.index
    
    # express numbers as arrays and reshape
    y = np.array(y)
    x = np.array(x)
    x = x.reshape(-1, 1)
    
    # make sure indices match up
    y = y[x[:,0].argsort()]
    x = x[x[:, 0].argsort()]

    # plot figure
    plt.figure(figsize=(16, 8))

    # standard linear regression
    linreg = LinearRegression()
    linreg.fit(x, y)

    # 2nd degree polynomial regression
    poly2 = PolynomialFeatures(degree=2)
    x_poly2 = poly2.fit_transform(x)
    poly_reg2 = LinearRegression()
    poly_reg2.fit(x_poly2, y)

    # third degree polynomial regression 
    poly3 = PolynomialFeatures(degree=3)
    x_poly3 = poly3.fit_transform(x)
    poly_reg3 = LinearRegression()
    poly_reg3.fit(x_poly3, y)

    # predict on x values
    pred = linreg.predict(x)
    pred2 = poly_reg2.predict(x_poly2)
    pred3 = poly_reg3.predict(x_poly3)

    # plot regression lines
    plt.scatter(x, y)
    plt.yscale('log')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Average')
    plt.plot(x, pred, c='red', label='Linear regression line')
    plt.plot(x, pred2, c='yellow', label='Polynomial regression line 2')
    plt.plot(x, pred3, c='#a3cfa3', label='Polynomial regression line 3');

In [ ]:
# group by average month sold mean to see relationship
y = train_data.groupby('latest_salemonth')['price'].mean()
plot_polys(y, "Month", "Month Sold Mean")

In [ ]:
# adding our chosen polynomial features

def create_polynomial_array(data, column, num_features):
    values = data[column]
    poly_array = np.array(values)
    poly_array = poly_array.reshape(-1,1)
    poly_fit = PolynomialFeatures(degree=num_features, include_bias=False)
    fit_features = poly_fit.fit_transform(poly_array)
    poly_df = pd.DataFrame(fit_features)
    return poly_df

month_poly = create_polynomial_array(df_cont_low_one_hot, 'month_smooth',2)

df_cont_low_one_hot['month1'] = month_poly[1]


### NLP 

In [ ]:
nlp = ['sentences']

nlp_train = train_data[nlp]

In [ ]:
nlp_train.head(10)

In [ ]:
v = TfidfVectorizer(sublinear_tf=True, max_df=0.9, min_df=.005, ngram_range=(1,4), max_features=1000)
x = v.fit_transform(nlp_train['sentences'])

train_word_vectors = pd.DataFrame(x.toarray(), columns=v.get_feature_names())

In [ ]:
predictors_train = sm.add_constant(train_word_vectors)
model = sm.OLS(y_train, predictors_train).fit()
model.summary()

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = included[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included



In [ ]:
# this was for ngrams of 1, 4 with 1000 returns

#important_ngrams = stepwise_selection(train_word_vectors, y_train, verbose=True)

important_ngrams = ['zilker', 'design', 'view', 'wine', 'outdoor', 'main', 'laminate', 'barton', 'spa', 'great', 'heart', 'finish', 'i35', 'paint', 'hill', 'meadow', 'classic', 'chef kitchen', 'airport', 'lake', 'community', 'pool', 'guest', 'gourmet kitchen', 'default', 'office', 'custom', 'congress', 'luxury', 'flat', 'hardwood', 'build', 'vinyl', 'suite', 'courtyard', 'original', 'travis', 'medium', 'marble', 'level', 'walkable', 'tile', 'condo', 'west', 'modern', 'playground', 'addition', 'construction', 'south', 'detach', 'northwest', 'entertainer', 'oak', 'washer', 'central', 'character', 'city', 'investment', 'park pool', 'study', 'wall', 'garden tub', 'quality', 'fan', 'mckinney', 'height', 'north', 'granite counter', 'shed', 'indoor', 'branch', 'isd', 'community pool', 'concept', 'floor', 'acre', 'major', 'prime', 'anderson', 'luxurious', 'downstairs', 'line', 'contemporary', 'ut', 'stunning', 'wet bar', 'ground', 'easy access', 'price', 'convenient', 'minute', 'upgrade include', 'mile downtown', 'adorable', 'waterfall', 'pre', 'patio', 'en', 'en suite', 'screen', 'circle', 'like', 'plenty', 'car', 'main living', 'block', 'east', 'unique', 'replace', 'tankless', 'space', 'breakfast', 'staircase', 'new', 'window', 'playroom', 'yes', 'amenity', 'restaurant', 'canyon']

#print(important_ngrams)

['zilker', 'design', 'view', 'wine', 'outdoor', 'main', 'laminate', 'barton', 'spa', 'great', 'heart', 'finish', 'i35', 'paint', 'hill', 'meadow', 'classic', 'chef kitchen', 'airport', 'lake', 'community', 'pool', 'guest', 'gourmet kitchen', 'default', 'office', 'custom', 'congress', 'luxury', 'flat', 'hardwood', 'build', 'vinyl', 'suite', 'courtyard', 'original', 'travis', 'medium', 'marble', 'level', 'walkable', 'tile', 'condo', 'west', 'modern', 'playground', 'addition', 'construction', 'south', 'detach', 'northwest', 'entertainer', 'oak', 'washer', 'central', 'character', 'city', 'investment', 'park pool', 'study', 'wall', 'garden tub', 'quality', 'fan', 'mckinney', 'height', 'north', 'granite counter', 'shed', 'indoor', 'branch', 'isd', 'community pool', 'concept', 'floor', 'acre', 'major', 'prime', 'anderson', 'luxurious', 'downstairs', 'line', 'contemporary', 'ut', 'stunning', 'wet bar', 'ground', 'easy access', 'price', 'convenient', 'minute', 'upgrade include', 'mile downtown', 'adorable', 'waterfall', 'pre', 'patio', 'en', 'en suite', 'screen', 'circle', 'like', 'plenty', 'car', 'main living', 'block', 'east', 'unique', 'replace', 'tankless', 'space', 'breakfast', 'staircase', 'new', 'window', 'playroom', 'yes', 'amenity', 'restaurant', 'canyon']

In [ ]:
# this was for ngrams of 2, 4 with 600 returns

#important_ngrams = stepwise_selection(train_word_vectors, y_train, verbose=True)

#important_ngrams = ['zilker park', 'chef kitchen', 'gourmet kitchen', 'main house', 'outdoor kitchen', 'main level', 'barton creek', 'pool spa', 'master suite', 'outdoor living', 'high end', 'ceiling window', 'easy access', 'hardwood floor', 'wet bar', 'custom build', 'garden tub', 'granite counter', 'southpark meadow', 'bird lake', 'hill country', 'custom cabinetry', 'en suite', 'south congress', 'spa like', 'laminate flooring', 'washer dryer', 'ceiling fan', 'major employer', 'wall window', 'new construction', 'south park', 'tile flooring', 'fresh paint', 'community pool', 'great location', 'large backyard', 'attention detail', 'outdoor space', 'laminate floor', 'mile downtown', 'gas fireplace', 'laminate wood', 'wood floor', 'brand new', 'investment opportunity', 'acre lot', 'walnut creek', 'walk distance', 'mid century', 'tankless water', 'vinyl plank', 'great open', 'double oven', 'sys yes', 'floor ceiling', 'tile backsplash', 'new roof', 'storage shed', 'onion creek', 'hyde park', 'price sell', 'custom cabinet', 'mopac i35', 'major highway', 'hard tile', 'stainless steel', 'quiet neighborhood', 'floor large', 'picture window', 'floor master', 'open living', 'backyard perfect', 'glass door', 'upgrade include', 'office space', 'location close', 'award win', 'park pool', 'ceramic tile', 'screen porch', 'wood burn', 'soak tub', 'open concept', 'concept live', 'new carpet', 'new flooring', 'yard large', 'large window', 'mature oak', 'water heater', 'pool playground', 'upstairs game', 'cover patio', 'floor open', 'quick access', 'tall ceiling', 'wood look', 'creek greenbelt', 'default listing']

#print(important_ngrams)

In [ ]:
'''model = LinearRegression()
model.fit(train_word_vectors, y_train)

from sklearn.inspection import permutation_importance
r = permutation_importance(model, train_word_vectors, y_train,
                           n_repeats=10,
                            random_state=0,
                          n_jobs=-1)

importances = {}

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] >= 0.001:
        importances[train_word_vectors.columns[i]] = r.importances_mean[i]
    else: continue
        
importances

important_ngrams = list(importances.keys())
important_ngrams '''

In [ ]:
important_ngrams = ['en suite',
 'en',
 'quiet cul de',
 'quiet cul',
 'sys yes',
 'sprinkler sys yes',
 'de',
 'cul de',
 'de lot',
 'cul de lot',
 'default',
 'pool',
 'main',
 'zilker',
 'steel',
 'cul',
 'natural light',
 'stainless steel',
 'large corner lot',
 'mid century',
 'award win',
 'barton',
 'washer dryer',
 'large corner',
 'natural',
 'floor',
 'wine',
 'heart',
 'award',
 'corner lot',
 'design',
 'corner',
 'washer',
 'view',
 'finish',
 'steel appliance',
 'swimming',
 'round rock',
 'garden tub separate',
 'hill country',
 'brand new',
 'easy access',
 'south',
 'open concept',
 'hill',
 'new',
 'energy efficient',
 'brand',
 'ton natural light',
 'distance',
 'lake',
 'hike bike',
 'walk distance',
 'country',
 'curb appeal',
 'year old',
 'level',
 'luxury',
 'dryer',
 'condo',
 'vault ceiling',
 'community',
 'chef kitchen',
 'appliance',
 'isd',
 'pane',
 'minute',
 'park pool',
 'curb',
 'space',
 'old',
 'formal',
 'living',
 'new construction',
 'vault',
 'walk closet',
 'floor plan',
 'classic',
 'lot natural light',
 'nest',
 'nest thermostat',
 'marble',
 'meadow',
 'recess lighting',
 'convenient',
 'courtyard',
 'tub separate',
 'green space',
 'main floor',
 'congress',
 'walkable',
 'flat',
 'sys',
 'bike trail',
 'main living',
 'double pane',
 'lot natural',
 'master suite',
 'century',
 'vinyl',
 'addition',
 'conveniently locate',
 'hardwood',
 'guest',
 'character',
 'miss',
 'garden tub separate shower',
 'formal living',
 'west',
 'swimming pool',
 'main level',
 'energy',
 'subway',
 'medium',
 'efficient',
 'sport',
 'ton natural',
 'ut',
 'living area',
 'access mopac',
 'gas fireplace',
 'downstairs',
 'formal dining',
 'recess',
 'laminate',
 'mid',
 'steiner ranch',
 'stone fireplace',
 'creek greenbelt',
 'tennis court',
 'access',
 'build',
 'community pool',
 'interior exterior paint',
 'fresh',
 'fan',
 'outdoor kitchen',
 'airport',
 'sport court',
 'double',
 'tile',
 'investment',
 'walnut creek',
 'amenity',
 'elementary',
 'contemporary',
 'mckinney',
 'great',
 'elementary school',
 'ground',
 'garden tub',
 'ih35',
 'entertainer',
 'car',
 'detach',
 'parking',
 'south congress',
 'east',
 'indoor',
 'round',
 'replace',
 'like',
 'city',
 'adorable',
 'bamboo',
 'i35',
 'restaurant',
 'quartz',
 'mother',
 'soak tub',
 'office',
 'upgrade',
 'large',
 'yes',
 'quality',
 'set',
 'builder',
 'northwest',
 'green',
 'breakfast bar',
 'height',
 'tankless',
 'exterior paint',
 'north',
 'vinyl plank',
 'branch',
 'outdoor living',
 'paint',
 'large backyard',
 'easy access mopac',
 'light',
 'stone',
 'lot',
 'luxurious',
 'large walk closet',
 'gourmet kitchen',
 'warranty',
 'living kitchen',
 'tennis',
 'walnut',
 'travertine',
 'hoa',
 'subway tile',
 'concept']

In [ ]:
# Run our linear regression again, using only the features recommended by our feature selector

train_word_vectors_refined = train_word_vectors[important_ngrams]

predictors_int = sm.add_constant(train_word_vectors_refined)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

## Process Test Set

### Categoricals

In [ ]:
holdout_categoricals = holdout[categories]

# binning our year built bins
holdout_categoricals["year_block"], year_bins = pd.qcut(holdout_categoricals['yearBuilt'], q=num_bins, retbins=True, labels=labels)

holdout_categoricals.drop('yearBuilt', axis=1, inplace=True)

# telling Pandas that these columns are categoricals
for item in holdout_categoricals.columns:
    holdout_categoricals[item] = holdout_categoricals[item].astype('category')

# make a processed bins file for use with linear regression
df_cats_high_one_hot_holdout = pd.get_dummies(holdout_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=True)
df_cats_low_one_hot_holdout = pd.get_dummies(holdout_categoricals[low_one_hot_cat], prefix=low_one_hot_cat, drop_first=True)

### Continuous

In [ ]:
# apply target encoding to test data, using train data to map

# create smooth additive encoded variables for zipcode, year built, and monthsold
holdout['zip_smooth'] = calc_smooth_mean(train_data, 'zipcode', 'price', zip_samples, holdout)
holdout['year_smooth'] = calc_smooth_mean(train_data, 'yearBuilt', 'price', 300, holdout)
holdout['month_smooth'] = calc_smooth_mean(train_data, 'latest_salemonth', 'price', month_samples, holdout)

# Create a wider lat and long zone to calculate an area mean
holdout['lat_zone'] = round(holdout['latitude'], 2)
holdout['long_zone'] = round(holdout['longitude'], 2)

# calculate smooth mean variables for lat and long, then create an interactive variable describing both together
holdout['lat_smooth'] = calc_smooth_mean(train_data, 'lat_zone', 'price', lat_samples, holdout)
holdout['long_smooth'] = calc_smooth_mean(train_data, 'long_zone', 'price', long_samples, holdout)
holdout['lat_long'] = round(np.sqrt(holdout['lat_smooth'] * holdout['long_smooth']), 0)

In [ ]:
holdout['year_smooth'].isna().sum()

In [ ]:
holdout.loc[holdout['year_smooth'].isna(), 'year_smooth'] = train_data['year_smooth'].mean()

In [ ]:
holdout_continuous = holdout[continuous]

In [ ]:
scaled_holdout_continuous = pd.DataFrame(scaler.transform(holdout_continuous),columns = holdout_continuous.columns)

# making our two continuous sets
df_cont_high_one_hot_holdout = scaled_holdout_continuous[high_one_hot_cont]
df_cont_low_one_hot_holdout = scaled_holdout_continuous[low_one_hot_cont]

In [ ]:
# adding polynomial features

month_poly = create_polynomial_array(df_cont_low_one_hot_holdout, 'month_smooth',2)

df_cont_low_one_hot_holdout['month1'] = month_poly[1]

### NLP

In [ ]:
nlp_holdout = holdout[nlp]

In [ ]:
x = v.transform(nlp_holdout['sentences'])

holdout_word_vectors = pd.DataFrame(x.toarray(), columns=v.get_feature_names())

holdout_word_vectors_refined = holdout_word_vectors[important_ngrams]

## Create Train/Test Final Set

In [ ]:
# make our train sets for one-hot encoded and target-encoded categoricals
X_train_onehot = pd.concat([df_cont_high_one_hot, df_cats_high_one_hot, train_word_vectors_refined], axis=1)
X_train_encoded = pd.concat([df_cont_low_one_hot, df_cats_low_one_hot, train_word_vectors_refined], axis=1)

# make our test sets for one-hot encoded and target-encoded categoricals
X_test_onehot = pd.concat([df_cont_high_one_hot_holdout, df_cats_high_one_hot_holdout, holdout_word_vectors_refined], axis=1)
X_test_encoded = pd.concat([df_cont_low_one_hot_holdout, df_cats_low_one_hot_holdout, holdout_word_vectors_refined], axis=1)

# make our target variable train and test sets, after log transforming our target variable
target = 'price' # target variable
y = np.log(df[target]) # our log-transformed target variable

y_train, y_test = train_test_split(y, test_size=0.2, random_state=randomstate) #
test_actual = np.exp(y_test)

y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

### Data Sets Reference

Our final data sets include:

* X_train_onehot, X_test_onehot - train/test split predictors for one-hot sets
* X_train_encoded, X_test_encoded - train/test split predictors for encoded sets
* y_train, y_test - target values for all sets
* y - log transformed price
* test_actual - exponentiated y_test prices

# Model Explorations

We're going to evaluate a few different variations of our linear regression model, as well as a few more complex model types. In order to keep track of our results, we'll be making a dictionary to store our model accuracy results.

In [ ]:
# prepare dictionary to store results
models = {}
models['Models'] = []
models['r2'] = []
models['mae'] = []
models['mae_500'] = []

values = pd.DataFrame({'actual':test_actual})

Our final data sets include:

* x_final_one_hot - Predictors with one-hot encoding for zipcode, month and year
* x_final_encoded - Predictors with additive smoothed encoding for zipcode, month and year
* y - log transformed price


* X_train_onehot, X_test_onehot - train/test split predictors for one-hot sets
* X_train_encoded, X_test_encoded - train/test split predictors for encoded sets
* y_train, y_test - target values for all sets
* test_actual - exponentiated y_test prices

## Picking our Base Features

We can use residuals plots to determine if features are important enough to add to our model. If we regress our target on a predictor, and then plot those residuals against a DIFFERENT predictor, our plot will tell us if the new feature might add to our model.

We're planning to one-hot encode our zips, but we'll use our zip_median continuous variable for now to start on.

We're going to add features in order of their correlation with price on our correlation heat map, so our base feature is zip code because it has the strongest correlation.

We regress our target on zip code, then we plot our residuals against total square footage.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['livingAreaSqFt']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Total Square Footage')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Square Footage - Response is Median_Zip')


plt.scatter(z, residuals, label="sample");

Regression on median zip, this time residuals plotted against school rating. This may not have the strong relationship we expected.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['avgSchoolRating']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Average School Rating')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Average School Rating - Response is Median Zip Code')


plt.scatter(z, residuals, label="sample");

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['lotSizeSqFt']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Lot Size Square Feet')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Lot Size Square Feet - Response is Median_Zip')

plt.scatter(z, residuals, label="sample");

Interesting how past a certain lot size, larger rounded numbers are used instead of specific numbers. There does seem to be a relationship here.

In [ ]:
x, y = np.array(df['median_zip']).reshape(-1,1), df['price']
z = np.array(df['numOfBedrooms']).reshape(-1,1)

model = LinearRegression()
model.fit(x, y)

test_predictions = model.predict(x)

residuals = y - test_predictions

fig = plt.figure(figsize=(15,10))

# Add labels for x and y axes
plt.xlabel('Number of Bedrooms')
plt.ylabel('Residuals')

# Add a title for the plot
plt.title('Residuals vs Number of Bedrooms - Response is Median_Zip')

plt.scatter(z, residuals, label="sample");

If we can see a pattern when we plot residuals vs a different predictor, it can tell us if a feature might add value to our model.

## Linear Regressions

### Basic Model Top Features Only

We're going to build our most baseline model using only the top three features -
    
    * zipcode
    * avgSchoolRating
    * livingAreaSqFt
    * numOfBathrooms

We can identify top features from our correlation heat map. Here's a reminder of the top:

#### One-Hot Encoded Categoricals

In [ ]:
# put together our basic feature set and preprocess

# one-hot encode categorical
base_cat = pd.DataFrame()
base_cat['zipcode'] = df['zipcode']
base_cat['zipcode'] = base_cat['zipcode'].astype('category')
base_cat_processed = pd.get_dummies(base_cat['zipcode'], prefix='zipcode', drop_first=True)
base_cat_processed.reset_index(inplace=True)
base_cat_processed.drop('index', axis=1, inplace=True)

# log transform and standard scale our continuous
base_cont = df[['avgSchoolRating', 'livingAreaSqFt', 'numOfBathrooms']]
#base_cont = np.log(base_cont)

scaler = StandardScaler()
base_cont_processed = pd.DataFrame(scaler.fit_transform(base_cont),columns = base_cont.columns)

#join cat and cont into predictor data frame
#x_base_set = pd.concat([base_cont_processed, base_cat_processed], axis=1)
x_base_set = base_cont_processed.join([base_cat_processed], how='inner') 

# train/test split
x_base_train, x_base_test = train_test_split(x_base_set, test_size=0.2, random_state=randomstate)

In [ ]:
# run model for R^2 score

model = LinearRegression()
model.fit(x_base_train, y_train)
cv_5 = cross_val_score(model, x_base_train, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(x_base_test)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions).astype(int)
test_predictions_unscaled = test_predictions_unscaled.flatten()

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);
# Residuals plot

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(np.exp(y_test), hist=True, kde=False)
sns.distplot(test_predictions_unscaled, hist=True, kde=False)
plt.title("Predictions vs Actual")
plt.legend(labels=['Actual Values of Price', 'Predicted Values of Price'])
plt.xlim(0,1500000);

In [ ]:
# append our results to our lists

models['Models'].append('Bare Bones Features LR - One-Hot Zip')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['lin_pred'] = test_predictions_unscaled

Our baseline model has an R^2 of 70% on only a few features. Our MAE is pretty high. We will see if we can improve on that with some other feature selection methods, and even some other model types.

#### Target Encoded Categoricals

In [ ]:
# put together our basic feature set and preprocess

# log transform and standard scale our continuous
base_cont = train_data[['avgSchoolRating', 'livingAreaSqFt', 'numOfBathrooms', 'zip_smooth']]
scaler = StandardScaler()
x_base_train = pd.DataFrame(scaler.fit_transform(base_cont),columns = base_cont.columns)


test_cont = holdout[['avgSchoolRating', 'livingAreaSqFt', 'numOfBathrooms', 'zip_smooth']]
scaler = StandardScaler()
x_base_test = pd.DataFrame(scaler.fit_transform(test_cont),columns = test_cont.columns)

In [ ]:
# run model for R^2 score

model = LinearRegression()
model.fit(x_base_train, y_train)
cv_5 = cross_val_score(model, x_base_train, y_train, cv=5)
r2 = cv_5.mean()
r2

Our R-squared of 60.5% is much lower than when we used our zip code as categoricals.

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(x_base_test)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions).flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);
# Residuals plot

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# append our results to our lists

models['Models'].append('Bare Bones Features LR - Encoded Zip')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['lin_pred'] = test_predictions_unscaled

### Linear Regression Model - ALL Features

#### One Hot Set

Run a base model with no cross-validation or specific feature selection with ALL possible features. We're going to use our one-hot encoded set which performed better in our first test.

In [ ]:
predictors_train = sm.add_constant(X_train_onehot)
model = sm.OLS(y_train, predictors_train).fit()
model.summary()

There is a multicollinearity of features in our feature set somewhere. Let's check.

The correlations it picked up are circumstancial.

There are a good number of features included in this model with a p-value over .05, meaning there is a greater than 5% chance that the results are due to randomness of the sample rather than the feature. A lot of our features have a very low p-value which indicates a very low chance that these results are not affected by the feature. 

Now we perform cross-validation with our base model over 5 splits and get our mean R^2.

In [ ]:
model = LinearRegression()
model.fit(X_train_onehot, y_train)
cv_5 = cross_val_score(model, X_train_onehot, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# How many predictors are in our base model?
print("{} predictors used for this model".format(X_train_onehot.shape[1]))

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(X_test_onehot)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions)
test_predictions_unscaled = test_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);

# Residuals plot

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# append our results to our lists

models['Models'].append('Basic LR - One-Hot')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['lin_pred'] = test_predictions_unscaled

##### Study Residuals

In [ ]:
# We need our statsmodels model again to plot residuals
predictors_train = sm.add_constant(X_train_onehot)
model = sm.OLS(y_train, predictors_train).fit()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "livingAreaSqFt", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "avgSchoolRating", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "lotSizeSqFt", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "numOfBedrooms", fig=fig)
plt.show()

#### Target Encoded Categoricals

Run a base model with no cross-validation or specific feature selection with ALL possible features. We're using our target categorical encoded set which performed worse in our first test.

In [ ]:
predictors_train = sm.add_constant(X_train_encoded)
model = sm.OLS(y_train, predictors_train).fit()
model.summary()

In [ ]:
model = LinearRegression()
model.fit(X_train_encoded, y_train)
cv_5 = cross_val_score(model, X_train_encoded, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# How many predictors are in our base model?
print("{} predictors used for this model".format(X_train_encoded.shape[1]))

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(X_test_encoded)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions)
test_predictions_unscaled = test_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);

# Residuals plot

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# append our results to our lists

models['Models'].append('Basic LR - Encoded')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['lin_pred'] = test_predictions_unscaled

## Linear Regression - Feature Selectors

Feature selectors are different methods to help us pick which features we want to use in our model. In our example above where we used ALL predictors in our linear regression, several of our features had a p-value over .05, which indicates that there is more than a 5% chance that the changes attributed to that feature were actually by random chance. We want features where our p-value is below a threshold that we specify where we are reasonably confident that the feature is contributing to the model and not by random chance.

### Permutation Importance

In [ ]:
model = LinearRegression()
model.fit(X_train_onehot, y_train)

from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_train_onehot, y_train,
                           n_repeats=15,
                            random_state=0,
                          n_jobs=-1)

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] >= 0.001:
        print(f"{X_train_onehot.columns[i]:<8} "
            f"\t\tImportance: {r.importances_mean[i]:.3f} ")

In [ ]:
importances = {}

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] >= 0.001:
        importances[X_train_onehot.columns[i]] = r.importances_mean[i]
    else: continue
        
importances

important_features_again = list(importances.keys())
important_features_again  

In [ ]:
permutation_x_train = X_train_onehot[important_features_again]
permutation_x_test = X_test_onehot[important_features_again]

model = LinearRegression()
model.fit(permutation_x_train, y_train)
cv_5 = cross_val_score(model, permutation_x_train, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
test_predictions = model.predict(permutation_x_test)
test_predictions

# reverse log transform our predicted values
test_predictions_unscaled = np.exp(test_predictions)
test_predictions_unscaled = test_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - test_predictions_unscaled

fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_unscaled, residuals);

# Residuals plot

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, test_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# append our results to our lists

models['Models'].append('LR - Permutation Importance')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['lin_pred'] = test_predictions_unscaled

### Forward-Backward Selector

First we'll try a simple forward-backward feature selection model based on p-value, using a statsmodel OLS linear regression model. This selector starts with zero features, internally runs the model for each feature individually, and adds the lowest p-value feature to its list to include. It then runs the model again with the original feature included and tries adding each other feature individually. It will either add the next best feature under the threshold or remove an existing feature if it is no longer within the threshold. This process iterates until all features in the model are under the p-value threshold.

This model takes quite some time to run, so is commented out with the results replicated in markdown following.

In [ ]:

result = stepwise_selection(X_train_onehot, y_train, verbose=True)

#print('resulting features:', result)


#result = ['livingAreaSqFt', 'hasAssociation_1', 'zipcode_78704', 'avgSchoolRating', 'zipcode_78703', 'avgSchoolSize', 'numPriceChanges', 'zipcode_78731', 'propertyTaxRate', 'zipcode_78753', 'zipcode_78737', 'zipcode_78757', 'zipcode_78746', 'zipcode_78756', 'zipcode_78702', 'zipcode_78735', 'modern', 'zipcode_78732', 'zipcode_78747', 'zipcode_78723', 'zipcode_78751', 'lotSizeSqFt', 'year_block_30', 'year_block_29', 'year_block_28', 'zipcode_78744', 'zipcode_78705', 'zipcode_78722', 'zipcode_78759', 'patioporch_1', 'condo', 'great', 'numOfPhotos', 'zipcode_78726', 'design', 'latest_salemonth_12', 'latest_salemonth_11', 'latest_salemonth_10', 'year_block_27', 'latest_salemonth_9', 'numOfStories', 'zipcode_78750', 'avgSchoolDistance', 'zipcode_78701', 'zipcode_78728', 'latest_salemonth_8', 'latest_salemonth_7', 'zipcode_78745', 'hasSpa_1', 'year_block_26', 'zipcode_78730', 'washer', 'zipcode_78733', 'latest_salemonth_6', 'price', 'convenient', 'investment', 'walkable', 'garageSpaces', 'zipcode_78721', 'zipcode_78752', 'zipcode_78741', 'window', 'outdoor', 'congress', 'default', 'accessibility_1', 'spa', 'zipcode_78749', 'zipcode_78729', 'zipcode_78717', 'year_block_13', 'latest_salemonth_5', 'hill', 'luxury', 'wine', 'classic', 'waterfall', 'unique', 'tankless', 'floor', 'view', 'detach', 'travis', 'zilker', 'new', 'laminate', 'hasHeating_1', 'zipcode_78739', 'zipcode_78758', 'zipcode_78727', 'zipcode_78748', 'zipcode_78736', 'zipcode_78724', 'year_block_24', 'zipcode_78660', 'year_block_25', 'indoor', 'community', 'medium', 'fan']

In [ ]:
# Run our linear regression again, using only the features recommended by our feature selector

X_train_refined = X_train_onehot[result]
X_test_refined = X_test_onehot[result]

predictors_int = sm.add_constant(X_train_refined)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

In [ ]:
print("{} predictors used".format(len(result)))

In [ ]:
model = LinearRegression()
model.fit(X_train_refined, y_train)
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

cv_5 = cross_val_score(model, X_train_refined, y_train, cv=cv)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
test_predictions_refined = model.predict(X_test_refined)

# reverse log transform our predicted values
test_predictions_refined_unscaled = np.exp(test_predictions_refined)
test_predictions_refined_unscaled=test_predictions_refined_unscaled.flatten()

# get residuals
residuals = test_actual - test_predictions_refined_unscaled

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(test_predictions_refined_unscaled, residuals)

In [ ]:
# get mean absolute error
mae = round(mean_absolute_error(test_actual, test_predictions_refined_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, test_predictions_refined_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# append our results to our lists

models['Models'].append('Forw-Back Selector')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['forw_back_pred'] = test_predictions_refined_unscaled

### Recursive Feature Elimination with Cross Validation - Linear Regression

RFECV is a reverse forward-backward selector. It starts the model with all features in use then removes the weakest one, and iterates until the best feature set is found. It uses integrated cross-validation to determine the optimal set of features in the model with the best cross-validated score. We score on mean absolute error.

In [ ]:
# Using sklearn RFECV to perform integrated CV while picking the number of features
# picks the number of features itself

model = LinearRegression(n_jobs=-4)
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

rfecv = RFECV(estimator=model, step=1, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-4)

# fit model to train set
rfecv.fit(X_train_onehot, y_train)

# print optimal number of features
print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X_train_onehot.columns
dset['used'] = rfecv.support_

# make a list of the features used in the rfecv
rfecv_result = list(dset[(dset['used'] == True)]['attr'])

# Show the features that RFECV did not use
dset[dset['used']==False]

In [ ]:
# Run our linear regression again in statsmodels, using the features recommended by our feature selector

X_train_rfecv = X_train_onehot[rfecv_result]
X_test_rfecv = X_test_onehot[rfecv_result]

predictors_int = sm.add_constant(X_train_rfecv)
model = sm.OLS(y_train, predictors_int).fit()
model.summary()

In [ ]:
# getting the r2 score of our best feature set
r2 = model.rsquared
r2

RFECV still includes features with a p-value over .05. Overall though, accuracy is higher than other feature selection methods.


In [ ]:
plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('R2', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

plt.show()

In [ ]:
# predict on new data
rfecv_predictions = rfecv.predict(X_test_onehot)

rfecv_predictions_unscaled = np.exp(rfecv_predictions)
rfecv_predictions_unscaled = rfecv_predictions_unscaled.flatten().astype(int)

# get residuals
residuals = test_actual - rfecv_predictions_unscaled

#plot residuals 
fig = plt.figure(figsize=(20,15))
plt.scatter(rfecv_predictions_unscaled, residuals);

In [ ]:
# get mean absolute error

mae = round(mean_absolute_error(test_actual, rfecv_predictions_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, rfecv_predictions_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(np.exp(y_test), hist=True, kde=False)
sns.distplot(rfecv_predictions_unscaled, hist=True, kde=False)
plt.legend(labels=['Actual Values of Price', 'Predicted Values of Price'])
plt.xlim(0,1500000);

In [ ]:
models['Models'].append('RFECV')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['rfecv_pred'] = rfecv_predictions_unscaled

## K-Nearest Neighbors Model

K-Nearest Neighbors is more commonly used for classification. Its basic premise is to determine "what is this like" in making a prediction, by looking at other things that are close in value/type. We can pick how many neighbors it assesses to make a classification. As we will see, it doesn't work very well for this type of application (or, I've not tuned the hyperparameters properly and/or don't know how to use it well).

We're using our target encoded data set on this.

In [ ]:
mae_val = [] #to store mae values for different k

# checks mean absolute error scores on k from 1 to 25
for K in range(25):
    K = K+1
    
    # set up the KNN regressor
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train_encoded, y_train)  #fit the model
    pred=model.predict(X_test_encoded) #make prediction on test set
    error = mean_absolute_error(y_test,pred) #calculate rmse
    mae_val.append(error) #store mae values
    print('MAE value for k= ' , K , 'is:', error)
    
# gets optimal k-value based on score minimum
index_min = np.argmin(mae_val) + 1

# makes model and fits using optimal k
model = neighbors.KNeighborsRegressor(n_neighbors = index_min)
model.fit(X_train_encoded, y_train)  #fit the model

# Get R^2 with cv
scores = cross_val_score(model, X_train_encoded, y_train, scoring='r2', cv=5, n_jobs=-1, error_score='raise')
r2 = np.mean(scores)
r2

In [ ]:
#make prediction on test set
pred_knn = model.predict(X_test_encoded)
pred_knn = np.exp(pred_knn).flatten().astype(int)

# get residuals
residuals = test_actual - pred_knn

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(pred_knn, residuals)

In [ ]:
# get mean absolute error

mae = round(mean_absolute_error(test_actual, pred_knn), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, pred_knn)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(np.exp(y_test), hist=True, kde=False)
sns.distplot(pred_knn, hist=True, kde=False)
plt.legend(labels=['Actual Values of Price', 'Predicted Values of Price'])
plt.xlim(0,1500000);

In [ ]:
models['Models'].append('KNN')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['knn_pred'] = pred_knn

## Support Vector Regression

Support vector regression is a form of regression that allows us to define the acceptable error in our model and then finds the line that best fits the data, according to our specifications. This is really useful with something like housing price predictions, where we are ok with our prediction being within a certain dollar amount. SVR will attempt to get all of the predictions within that dollar amount when possible. This will result in a fit line that is different than a linear regression would have produced, but should result in a lower absolute error, which is a reasonable scoring metric for housing price predictions.

We're going to use sklearn's GridSearchCV to find the optimal hyperparameters to use with our SVM! Here are the parameters we are trying out:

* kernel: linear is parametric, and rbf is non-parametric. One of these should perform better. Our data is not totally normal, so it might be rbf.
* epsilon: This value is how much error we're ok with accepting without assigning a penalty to the model
* C: The error that we will accept from a point outside our epsilon

Our C and epsilon need to be in scale with our output variable, which is our log-transformed price.


In [ ]:
# Parameter Tuning
'''
param_grid = {'C' : [.5, 1, 3, 5]            
              }

svr = SVR(epsilon=.05, kernel='linear')
grid_search = GridSearchCV(svr, param_grid, scoring='neg_mean_absolute_error', cv=5)

grid_search.fit(X_train_onehot, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

In [ ]:
'''# Parameter Tuning

param_grid = {'epsilon' : [.05, .1, .5, 1]            
              }

svr = SVR(C=5, kernel='linear')
grid_search = GridSearchCV(svr, param_grid, scoring='neg_mean_absolute_error', cv=5)

grid_search.fit(X_train_onehot, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

The following cannot produce results with the above commented out, but results have been replicated in the markdown following.

In [ ]:
'''# Parameter Tuning

param_grid = {'kernel' : ['linear', 'rbf', 'poly'],
              'gamma' : ['scale', 'auto']
              }

svr = SVR(C=5, epsilon=.05, tol=.01, verbose=True)
grid_search = GridSearchCV(svr, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)

grid_search.fit(X_train_onehot, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

[LibSVM]Best parameters set found on train set: 

    {'gamma': 'auto', 'kernel': 'rbf'}

    Grid scores on train set:

    -0.146 (+/-0.003) for {'gamma': 'scale', 'kernel': 'linear'}
    -0.157 (+/-0.005) for {'gamma': 'scale', 'kernel': 'rbf'}
    -0.184 (+/-0.007) for {'gamma': 'scale', 'kernel': 'poly'}
    -0.146 (+/-0.003) for {'gamma': 'auto', 'kernel': 'linear'}
    -0.139 (+/-0.002) for {'gamma': 'auto', 'kernel': 'rbf'}
    -0.194 (+/-0.004) for {'gamma': 'auto', 'kernel': 'poly'}

In [ ]:
# setting up estimator with our optimal parameters
params = {'kernel' : 'rbf', 'C' : 5, 'epsilon' : .05, 'gamma':'auto'}
svr = SVR(**params, verbose=True, tol=.008)

# fitting our estimator to train data
svr.fit(X_train_onehot, y_train)

# getting R^2 with cv
cv_5 = cross_val_score(svr, X_train_onehot, y_train, cv=5, n_jobs=-1)
r2 = cv_5.mean()
r2

In [ ]:
'''# Parameter Tuning

param_grid = {'C' : [1,3,5],            
              }

svr = SVR(epsilon=.05, tol=.008, verbose=True, kernel='rbf', gamma = 'auto')
grid_search = GridSearchCV(svr, param_grid, verbose=10, scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)

grid_search.fit(X_train_onehot, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

[LibSVM]Best parameters set found on train set: 
    
    {'C': 5}
    
    Grid scores on train set:
    
    -0.142 (+/-0.003) for {'C': 1}
    -0.139 (+/-0.002) for {'C': 3}
    -0.139 (+/-0.002) for {'C': 5}

In [ ]:
# make new predictions on test
predictions_SVR = svr.predict(X_test_onehot)
predictions_SVR_unscaled = np.exp(predictions_SVR)

# get residuals
residuals = test_actual - predictions_SVR_unscaled

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(predictions_SVR_unscaled, residuals)

In [ ]:
# get mean absolute error

mae = round(mean_absolute_error(test_actual, predictions_SVR_unscaled), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, predictions_SVR_unscaled)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
# plot actual and predictions
plt.figure(figsize=(10,5))
sns.distplot(np.exp(y_test), hist=True, kde=False)
sns.distplot(predictions_SVR_unscaled, hist=True, kde=False)
plt.legend(labels=['Actual Values of Price', 'Predicted Values of Price'])
plt.xlim(0,1500000);

In [ ]:
models['Models'].append('SVR')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['svr_pred'] = predictions_SVR_unscaled

## Redo Data for Decision-Based Regressors

We're now going to work with some different model types that are entirely different from linear regression.

There's conflicting information on whether we should use one-hot encoding, or target encoding. We'll solve this by trying both and figuring out what works best for our data set.

### Decision Tree Regressors - One Hot

We need to slightly redo our one-hot encodings to not drop the first entry. We'll also make year_built into total one-hot encodings rather than bins.

In [ ]:
dummies_boost_cats_test = pd.get_dummies(holdout_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=False)
dummies_boost_cats_train = pd.get_dummies(df_categoricals[high_one_hot_cat], prefix=high_one_hot_cat, drop_first=False)

x_train_boost = pd.concat([df_cont_high_one_hot, dummies_boost_cats_train, train_word_vectors_refined], axis=1)
x_test_boost = pd.concat([df_cont_high_one_hot_holdout, dummies_boost_cats_test, holdout_word_vectors_refined], axis=1)

# redoing our y_train and y_test as non-log transformed
y = df[target] # our target variable

# creating our train/validation sets and our test sets
y_train, y_test = train_test_split(y, test_size=0.2, random_state=randomstate)

# reset indices to avoid index mismatches
y_train = pd.DataFrame(y_train)
y_train.reset_index(inplace=True)
y_train.drop('index', axis=1, inplace=True)

y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop('index', axis=1, inplace=True)

#### Random Forest

Random Forest is a decision tree model. It runs a large number of randomized decision trees and uses the averages of its results to make a prediction. It attempts to pick each tree at random without a directed or iterative plan.

We're going to do extensive parameter tuning to find the best hyperparameters for our random forest, tuning only one or two hyperparameters at a time. When we find an optimal hyperparameter we will add it to our model and tune a different parameter. This iterative approach allows us to test parameters in less time than a large GridSearchCV, although a larger grid search would prove more exhaustive to explore parameter interactions.

Here are the parameters we are trying out:

    max_depth: This determines how many tree nodes the estimator looks at before making a prediction. We don't know what is best here, so we are trying a large scale from 10-100 first and seeing what looks right.
    min_samples_split: The minimum number of samples required to split an internal node
    min_samples_leaf: The minimum number of samples required to be at a leaf node
    max_features: The number of features to consider when looking for the best split
    bootstrap: Whether bootstrap samples are used when building trees
    n_estimators: The number of trees in the forest

In [ ]:
# visualize changes to model score as it is tried on different max depths from 10 to 100, to get a starting point for max depth

from sklearn.model_selection import validation_curve
def ValidationCurve(estimator, predictors, target, param_name, hyperparam):
    
    train_score, test_score = validation_curve(estimator, predictors, 
                                               target, param_name, np.arange(10,110,10), 
                                               cv=5, scoring='r2', verbose=10, n_jobs=-4)
    Rsqaured_train = train_score.mean(axis=1)
    Rsquared_test= test_score.mean(axis=1)
    
    plt.figure(figsize=(10,5))
    plt.plot(np.arange(10,110,10), Rsqaured_train, color='r', linestyle='-', marker='o', label='Training Set')
    plt.plot(np.arange(10,110,10), Rsquared_test, color='b', linestyle='-', marker='x', label='Testing Set')
    plt.legend(labels=['Training Set', 'Testing Set'])
    plt.xlabel(hyperparam)
    plt.ylabel('R_squared')
    plt.title("R^squared for Max Depth on Train/Test")
    
ValidationCurve(RandomForestRegressor(), x_train_boost, y_train, 'max_depth', 'Maximum Depth')

In [ ]:
'''# Parameter Tuning

param_grid = {'max_depth' : [20, 40, 60, 80, 100],            
              }

model = RandomForestRegressor(criterion='mae', verbose=10, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set:

    {'max_depth': 20}

    Grid scores on train set:

    0.782 (+/-0.028) for {'max_depth': 20}
    0.782 (+/-0.027) for {'max_depth': 40}
    0.781 (+/-0.027) for {'max_depth': 60}
    0.781 (+/-0.025) for {'max_depth': 80}
    0.781 (+/-0.024) for {'max_depth': 100}

In [ ]:
'''# Parameter Tuning

param_grid = {'max_depth' : [15, 20, 25],            
              }

model = RandomForestRegressor(criterion='mae', verbose=10)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-7)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 
    {'max_depth': 20}

    Grid scores on train set:

    0.778 (+/-0.029) for {'max_depth': 15}
    0.782 (+/-0.028) for {'max_depth': 20}
    0.779 (+/-0.027) for {'max_depth': 25}

In [ ]:
'''# Parameter Tuning

param_grid = {'min_samples_split': [2, 5, 8]            
              }

model = RandomForestRegressor(criterion='mae', verbose=10, max_depth=20, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'min_samples_split': 2}

    Grid scores on train set:

    0.783 (+/-0.028) for {'min_samples_split': 2}
    0.782 (+/-0.027) for {'min_samples_split': 5}
    0.781 (+/-0.025) for {'min_samples_split': 8}

In [ ]:
'''#Parameter Tuning

param_grid = {'min_samples_split': [2, 3, 4]            
              }

model = RandomForestRegressor(criterion='mae', verbose=10, max_depth=20, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'min_samples_split': 2}

    Grid scores on train set:

    0.782 (+/-0.028) for {'min_samples_split': 2}
    0.780 (+/-0.024) for {'min_samples_split': 3}
    0.781 (+/-0.024) for {'min_samples_split': 4}

In [ ]:
'''# Parameter Tuning

param_grid = {'min_samples_leaf': [1, 3],
              'max_features': ['auto', 'sqrt']
              }

model = RandomForestRegressor(criterion='mae', max_depth=20, min_samples_split=2, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'max_features': 'auto', 'min_samples_leaf': 1}

    Grid scores on train set:

    0.782 (+/-0.023) for {'max_features': 'auto', 'min_samples_leaf': 1}
    0.780 (+/-0.025) for {'max_features': 'auto', 'min_samples_leaf': 3}
    0.747 (+/-0.028) for {'max_features': 'sqrt', 'min_samples_leaf': 1}
    0.724 (+/-0.030) for {'max_features': 'sqrt', 'min_samples_leaf': 3}

In [ ]:
'''# Parameter Tuning

param_grid = {
         'bootstrap': [True, False]
        }

model = RandomForestRegressor(criterion='mae', max_depth=20, min_samples_split=2, max_features='auto', min_samples_leaf=1, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'bootstrap': True}

    Grid scores on train set:

    0.781 (+/-0.025) for {'bootstrap': True}
    0.598 (+/-0.074) for {'bootstrap': False}

In [ ]:
'''# Parameter Tuning

param_grid = {
 'n_estimators' : [250, 500, 1000]
    }

model = RandomForestRegressor(criterion='mae', max_depth=20, min_samples_split=2, bootstrap=True, max_features='auto', min_samples_leaf=1, n_jobs=-4)

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'n_estimators': 250}

    Grid scores on train set:

    0.780 (+/-0.024) for {'n_estimators': 100}
    0.784 (+/-0.025) for {'n_estimators': 250}
    0.783 (+/-0.026) for {'n_estimators': 500}

In [ ]:
# set our regressor with optimal parameterss. I'm going to go with 500 on the estimators because they all seemed very close.
randomforest = RandomForestRegressor(criterion='mae',
                                     max_depth=20, 
                                     min_samples_split=2, 
                                     max_features='auto', 
                                     min_samples_leaf=1, 
                                     bootstrap=True, 
                                     n_estimators=250, 
                                     n_jobs=-4)

# fit random forest regressor to train
randomforest.fit(x_train_boost, y_train)

# check cross_val R^2
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
cv_5 = cross_val_score(randomforest, x_train_boost, y_train, cv=cv, n_jobs=-4)
r2 = cv_5.mean()
r2

In [ ]:
# apply our model to our test set and get predicted values
forest_predictions = randomforest.predict(x_test_boost)

# calculate residuals
residuals = test_actual - forest_predictions

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(forest_predictions, residuals)

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, forest_predictions), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, forest_predictions)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
models['Models'].append('Random Forest - One Hot')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['forest_pred'] = forest_predictions

#### XGBoost

This model is entirely different than linear regression. Gradient boosting uses decision trees to learn about outcomes, with trees being added to the model one at a time and existing trees in the model are not changed. Each successive tree tries to improve upon the predictions of the first one, with the weights of the various decision points being updated each time. Gradient boosting uses the residuals to improve its next tree prediction. Overall much more opaque of a process than linear regression.

Gradient Boosting performs best with optimal parameter tuning. We're going to use sklearn's GridSearchCV to find the optimal hyperparameters to use with our gradient booster! Here are the parameters we are trying out:

* n_estimators: Number of boosts to perform. Gradient boosting is pretty robust to over-fitting so more is usually better
* max_depth: This determines how many tree nodes the estimator looks at before making a prediction. We don't know what is best here, so we are trying things from 2-4 to see what works the best
* min_child_weight: Min sum of instance weight needed in a child
* gamma: Minimum loss reduction to make another partition on a leaf node. Higher results in more conservative algorithm.
* subsample: Ratio of training sets. .5 means that it will sample half the training data before making trees. Occurs with each boosting iteration.
* colsample_by_tree: ratio of columns when making a tree
* alpha: L1 regularization. Higher will make model more conservative.
* learning_rate: Tuning this setting alters how much the model corrects after it runs a boost. .1 is a common rate and we will test a lower and higher rate as well.

In [ ]:
# visualize changes to model score as it is tried on different max depths from 10 to 80, to get a starting point for max depth

from sklearn.model_selection import validation_curve
def ValidationCurve(estimator, predictors, target, param_name, hyperparam):
    
    train_score, test_score = validation_curve(estimator, predictors, target, 
                                               param_name, np.arange(10,90,10), 
                                               cv=5, scoring='r2', n_jobs=-4)
    Rsqaured_train = train_score.mean(axis=1)
    Rsquared_test= test_score.mean(axis=1)
    
    plt.figure(figsize=(10,5))
    plt.plot(np.arange(10,90,10), Rsqaured_train, color='r', linestyle='-', marker='o', label='Training Set')
    plt.plot(np.arange(10,90,10), Rsquared_test, color='b', linestyle='-', marker='x', label='Testing Set')
    plt.legend(labels=['Training Set', 'Testing Set'])
    plt.xlabel(hyperparam)
    plt.ylabel('R_squared')
    plt.title("R^squared for Max Depth on Train/Test")
    
ValidationCurve(xgb.XGBRegressor(), x_train_boost, y_train, 'max_depth', 'Maximum Depth')

In [ ]:
'''# Parameter Tuning max_depth

param_grid = {"max_depth": [5, 10],
                          
              }

model = xgb.XGBRegressor(
                 n_estimators=250,                                                                    
                 seed=42,
                 missing=0,
                 eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-4)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'max_depth': 5}

    Grid scores on train set:
    
    0.790 (+/-0.016) for {'max_depth': 5}
    0.770 (+/-0.037) for {'max_depth': 10}

In [ ]:
'''# Parameter Tuning max_depth and min_child_weight

param_grid = {"max_depth": [3, 5, 7],
              "min_child_weight" : [6, 8, 10]            
              }

model = xgb.XGBRegressor(
                 n_estimators=250,                                                                    
                 seed=42,
                 missing=0,
                 eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'max_depth': 3, 'min_child_weight': 8}

    Grid scores on train set:

    0.794 (+/-0.019) for {'max_depth': 3, 'min_child_weight': 6}
    0.797 (+/-0.025) for {'max_depth': 3, 'min_child_weight': 8}
    0.795 (+/-0.021) for {'max_depth': 3, 'min_child_weight': 10}
    0.787 (+/-0.018) for {'max_depth': 5, 'min_child_weight': 6}
    0.785 (+/-0.019) for {'max_depth': 5, 'min_child_weight': 8}
    0.792 (+/-0.022) for {'max_depth': 5, 'min_child_weight': 10}
    0.783 (+/-0.028) for {'max_depth': 7, 'min_child_weight': 6}
    0.780 (+/-0.023) for {'max_depth': 7, 'min_child_weight': 8}
    0.780 (+/-0.031) for {'max_depth': 7, 'min_child_weight': 10}

In [ ]:
'''# Parameter Tuning max_depth and min_child_weight

param_grid = {"max_depth": [2, 3, 4],
              "min_child_weight" : [7, 8, 9]            
              }

model = xgb.XGBRegressor(
                 n_estimators=250,                                                                    
                 seed=42,
                 missing=0,
                 eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'max_depth': 3, 'min_child_weight': 8}

    Grid scores on train set:

    0.794 (+/-0.019) for {'max_depth': 2, 'min_child_weight': 7}
    0.793 (+/-0.023) for {'max_depth': 2, 'min_child_weight': 8}
    0.795 (+/-0.024) for {'max_depth': 2, 'min_child_weight': 9}
    0.797 (+/-0.022) for {'max_depth': 3, 'min_child_weight': 7}
    0.797 (+/-0.025) for {'max_depth': 3, 'min_child_weight': 8}
    0.795 (+/-0.026) for {'max_depth': 3, 'min_child_weight': 9}
    0.790 (+/-0.016) for {'max_depth': 4, 'min_child_weight': 7}
    0.793 (+/-0.019) for {'max_depth': 4, 'min_child_weight': 8}
    0.792 (+/-0.029) for {'max_depth': 4, 'min_child_weight': 9}

In [ ]:
'''# Parameter Tuning gamma

param_grid = {'gamma':[.1, .3, .5, .7, .9]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'gamma': 0.1}

    Grid scores on train set:

    0.797 (+/-0.025) for {'gamma': 0.1}
    0.797 (+/-0.025) for {'gamma': 0.3}
    0.797 (+/-0.025) for {'gamma': 0.5}
    0.797 (+/-0.025) for {'gamma': 0.7}
    0.797 (+/-0.025) for {'gamma': 0.9}

In [ ]:
'''# Parameter Tuning subsample

param_grid = {
 'subsample':[.2, .4, .6, .8, 1],
 
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

In [ ]:
'''# Parameter Tuning colsample_by_tree

param_grid = {
 'colsample_bytree':[.2, .4, .6, .8, 1]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

In [ ]:
# Parameter Tuning alpha

param_grid = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 10, 100]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_lambda = 1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'reg_alpha': 10}

    Grid scores on train set:

    0.803 (+/-0.019) for {'reg_alpha': 1e-05}
    0.803 (+/-0.019) for {'reg_alpha': 0.01}
    0.803 (+/-0.019) for {'reg_alpha': 0.1}
    0.803 (+/-0.019) for {'reg_alpha': 1}
    0.803 (+/-0.019) for {'reg_alpha': 10}
    0.802 (+/-0.020) for {'reg_alpha': 100}

In [ ]:
'''# Parameter Tuning alpha

param_grid = {
    'reg_alpha':[5, 10, 25, 50]
    }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_lambda = 1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'reg_alpha': 10}

    Grid scores on train set:

    0.803 (+/-0.019) for {'reg_alpha': 5}
    0.803 (+/-0.019) for {'reg_alpha': 10}
    0.802 (+/-0.020) for {'reg_alpha': 25}
    0.802 (+/-0.020) for {'reg|_alpha': 50}

In [ ]:
# Parameter Tuning lambda

param_grid = {'lambda':[0.1, 1, 10, 100, 500, 1000, 2500]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 10,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'lambda': 1}

    Grid scores on train set:

    0.797 (+/-0.022) for {'lambda': 0.1}
    0.803 (+/-0.019) for {'lambda': 1}
    0.802 (+/-0.013) for {'lambda': 10}
    0.794 (+/-0.018) for {'lambda': 100}
    0.780 (+/-0.029) for {'lambda': 500}
    0.772 (+/-0.023) for {'lambda': 1000}
    0.755 (+/-0.021) for {'lambda': 2500}

In [ ]:
# Parameter Tuning lambda

param_grid = {'lambda':[0.5, 1, 5]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 10,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

Best parameters set found on train set: 

    {'lambda': 1}

    Grid scores on train set:

    0.799 (+/-0.023) for {'lambda': 0.5}
    0.803 (+/-0.019) for {'lambda': 1}
    0.801 (+/-0.023) for {'lambda': 5}

In [ ]:
'''# Parameter Tuning num_estimators

param_grid = {'num_estimators':[250, 500, 1000, 5000]            
              }

model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 10,
                reg_lambda = 1,
                seed=42,
                missing=0,
                eval_metric='mae' )

grid_search = GridSearchCV(model, param_grid=param_grid, cv = 5, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(x_train_boost, y_train)

grid_search.best_estimator_

print("Best parameters set found on train set: \n")
print(grid_search.best_params_)
print("\nGrid scores on train set:\n")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))'''

Best parameters set found on train set: 

    {'num_estimators': 250}

    Grid scores on train set:

    0.803 (+/-0.019) for {'num_estimators': 250}
    0.803 (+/-0.019) for {'num_estimators': 500}
    0.803 (+/-0.019) for {'num_estimators': 1000}
    0.803 (+/-0.019) for {'num_estimators': 5000}

In [ ]:
best_xgb_model = xgb.XGBRegressor(
                n_estimators=250,
                max_depth = 3,
                min_child_weight = 8,
                gamma = .1,
                colsample_bytree=.8,
                reg_alpha = 10,
                reg_lambda = 1,
                seed=42,
                missing=0,
                eval_metric='mae' )
best_xgb_model.fit(x_train_boost, y_train)

cv_5 = cross_val_score(best_xgb_model, x_train_boost, y_train, cv=5)
r2 = cv_5.mean()
r2

In [ ]:
# make prediction
preds = best_xgb_model.predict(x_test_boost)

#log residuals
residuals = test_actual - preds

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(preds, residuals)

In [ ]:
# Calculate our mean absolute error

mae = round(mean_absolute_error(test_actual, preds), 2)
mae

In [ ]:
# MAE for properties UNDER 500k

under500 = pd.DataFrame(list(zip(test_actual, preds)), columns =['actual', 'predictions'])
under500 = under500[under500['actual'] < 500000]
mae500 = round(mean_absolute_error(under500['actual'], under500['predictions']), 2)
mae500

In [ ]:
models['Models'].append('XGBoost')
models['r2'].append(r2)
models['mae'].append(mae)
models['mae_500'].append(mae500)
values['boost_pred'] = preds

### Decision Tree Regressors - Target Encoding

# Model Selection

We ran several different types of models, and logged the r^squared and mean absolute error for each model type. Which model performed the best for us?

In [ ]:
# make data frame from our models dictionary
model_types = pd.DataFrame(models)

# sort data frame by mae and reset index
model_types = model_types.sort_values('mae', ascending=True).reset_index()
model_types.drop('index',axis=1, inplace=True)
model_types.set_index('Models', inplace=True)

model_types

In [ ]:
# plot model mae

plt.figure(figsize=(15,10))
plt.plot(model_types['mae'])
plt.title("Mean Average Error")
plt.xticks(rotation=90)
plt.xlabel('Model')
plt.ylabel("MAE");

In [ ]:
# get residuals for a few types
values['rfecv_resid'] = (values['actual'] - values['rfecv_pred'])/10000
values['compsmodel_resid'] = (values['actual'] - values['compsmodel_pred'])/10000
values['boost_resid'] = (values['actual'] - values['boost_pred'])/10000

In [ ]:
# plot overlapping residuals for a few types
fig = plt.figure(figsize=(20,15))

plt.scatter(values['boost_pred'], values['boost_resid'], color='red')
plt.scatter(values['compsmodel_pred'], values['compsmodel_resid'], color='blue')
plt.scatter(values['rfecv_pred'], values['rfecv_resid'], color='green')
plt.title("Residuals")
#plt.xticks(rotation=90)
plt.xlabel('Predictions in millions')
plt.ylabel("Distance from Actual, in $10,000");

Ultimately we will select the sklearn RFECV method for our model. This method performs optimal feature selection via cross validation in order to determine the best feature set for the data. 

Support Vector Regression and Ridge performed better, but it doesn't fulfill two of the asks for this project - that the model be a linear regression, and that it be easily explained. RFECV also had the lowest MAE on properties under 500k, so we're happy with its overall performance.

# Final Model

Now that we've chosen a favorite regression for this problem (RFECV), we want to be able to actually use the model to predict new data. Our steps are as follows:

* Build a model using our ENTIRE dataset for deployment
* Write our own standardization functions that we can apply to incoming new data
* Prepare a more intuitive GUI for predicting on new data

## Prepare full data set for final model

In [ ]:
# merge our x_test and x_train continuous variables into one data frame, then make sure index is reset
X_all_continuous = x_test_cont.append(x_train_cont)
X_all_continuous.reset_index(inplace=True)
X_all_continuous.drop(['index'], axis=1, inplace=True)

# merge our x_test and x_train categorical variables into one data frame, then make sure index is reset
X_all_categoricals = x_test_cat.append(x_train_cat)
X_all_categoricals.reset_index(inplace=True)
X_all_categoricals.drop('index', axis=1, inplace=True)

While forming our model, we used the sklearn StandardScaler. But how do we scale brand new features that we are trying to predict from? In order to do so, we need to write a standardization function for each of our features, that we can then save and apply to our new prediction data.

In [ ]:
# make dictionary to store standardization coefficients
standardization_coeffs = {}

# make standardization coefficients for each column
for item in X_all_continuous:
    standardization_coeffs[item+'_mean'] = X_all_continuous[item].mean()
    standardization_coeffs[item+'_std'] = X_all_continuous[item].std()

# log transform all continuous
X_all_continuous = np.log(X_all_continuous)

# apply standardization to all continuous
for item in X_all_continuous:
    X_all_continuous[item] = (X_all_continuous[item] - standardization_coeffs[item+'_mean'])/standardization_coeffs[item+'_std']
    
# concat continuous and categoricals into a final X
X_final = pd.concat([X_all_continuous, X_all_categoricals], axis=1)

# concat train and test y into a final y, reset index so it matches X
y_final = y_test.append(y_train_val)
y_final.reset_index(inplace=True)
y_final.drop('index',  axis=1, inplace=True) 

## Train Final Model

Using sklearn RFECV to perform integrated CV while picking the number of features.

We're running a heavy cv to ensure that we really, truly have the right feature selection. We're using sets of 5 folds, and then repeating 5 times with different sets of 5 folds using mean absolute error as the scoring metric - we're looking for lowest error range

In [ ]:
# define estimator and cv plan
model = LinearRegression()
cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)

# create final model using RFECV
rfecv = RFECV(estimator=model, step=1, cv=cv, scoring='neg_mean_absolute_error')

# fit final model to all data
rfecv.fit(X_final, y_final)

# final model r^2
rfecv.score(X_final, y_final)

### Get Coefficient Importance

In [ ]:
print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
# get list of features used by the final model
dset = pd.DataFrame()
dset['attr'] = X_final.columns
dset['used'] = rfecv.support_

# make a list of the features used in the rfecv
rfecv_result = list(dset[(dset['used'] == True)]['attr'])

In [ ]:
# Run the model with the selected features in statsmodels OLS so we can get
# a list of model pvalues
X_specific = X_final[rfecv_result]

predictors_train = sm.add_constant(X_specific)
model = sm.OLS(y_final, predictors_train).fit()

model.summary()

We have values over p-value .05, so we're going to dump those by doing the feature selector on this.

In [ ]:
# run feature selector and get list of features with p-value under .05
stepwise_result = stepwise_selection(X_specific, y_final, verbose=True)

Our stepwise selector shows us our most important features. Grade, sqft_living, condition, and several zip codes were incredibly important features. Almost all included features had p-values so low that the probability that these features are influencing the price by random chance is near zero. Our highest included p-value, zipcode 98003, has only a .3% chance of having influenced prices by random chance.

In [ ]:
# make one last data set using the features that the selector accepted
X_specific = X_specific[stepwise_result]

# make final model
final_model = LinearRegression()
final_model.fit(X_specific, y_final)

# evaluate final model
predictors_train = sm.add_constant(X_specific)
model = sm.OLS(y_final, predictors_train).fit()

model.summary()

In [ ]:
# save our final summary to image
plt.rc('figure', figsize=(10,23))
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
#plt.savefig('images/output.png')

In [ ]:
# final model MAE

end_z = np.exp(final_model.predict(X_specific))
mae = round(mean_absolute_error(np.exp(y_final), end_z), 2)
mae

In [ ]:
# get residuals
residuals = np.exp(y_final) - end_z

# plot residuals
fig = plt.figure(figsize=(20,15))
plt.scatter(end_z, residuals)
plt.title("Residuals")
#plt.xticks(rotation=90)
plt.xlabel('Predictions in millions')
plt.ylabel("Distance from Actual, in $10,000");

#plt.savefig('images/residuals.png')

Our residuals show the heteroscedasticity described by our Durbin Watson score above 2.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True, ax=ax)

QQ-plot of residuals shows heavy tails.

### Observations on final model summary

> 88.2% of our variation in price can be explained by our model variables. Our equal R^2 and adjusted R^2 tells us that all of our features are contributing to our model. Adjusted R^2 penalizes the R^2 formula based on number of variables, so a same score tells us that our variables are all contributors.

> F-statistic and Prob(F-statistic) tell us if our variable group as a whole is statistically significant.The null hypothesis is a model with no variables (only intercept). Our prob(f-statistic) evaluates the chance that our null hypothesis is true and our variables have no effect. On our model this chance is so low that it's functionally 0. F-statistic is evaluating all of the coefficients used together, so p-value of individual coefficients is still important. So this number tells us that our MODEL is significant without telling us if any individual feature is significant.

> Omnibus and prob(omnibus) are telling us that our residuals are not normally distributed. We want to see a value close to 0 for Omnibus, and a value close to 1 for Prob(Omnibus). Neither of these things is the case. We saw heteroscedasticity on the residuals for all of our model types that we tried. All of our models seem to perform well at lower values and then variance increases as price increases. This was true on both linear and nonlinear approaches such as decision trees.

> Our data is skewed slightly left. Our data has high kurtosis which implies tighter grouping of our residuals around zero and fewer outliers, which implies a fitted model.

> Durbin Watson is a measure of homoscedasticity - an even distribution of errors. We are showing slight heteroscedasticity because we'd like to see this number between 1 and 2. We saw that heteroscedasticity in the residuals.

> Our low condition number implies no multicollinearity with our features.

> Overall our characteristics are mediocre. Our high omnibus and above 2 Durbin-Watson are problematic. Our QQ-plot shows very heavy tails. But no better model was found to correct for the heteroscedasticity.



# Analysis

> Our final model utilizes a combination of continuous variables and one-hot-encoded categoricals to produce a linear regression with R^2 of 88.2% and a mean absolute error of 56k. I tried several different transformations including polynomial features, mean target encoding, lower-granularity binning, and median rank as a continuous, and ALL of these efforts resulted in a lower R^2 and higher mean absolute error, leading to a final decision to one-hot encode all 70 zip codes individually. Similar efforts on other categoricals such as age and month sold also did not improve the model over one-hot encoding. This resulted in the greatest accuracy despite a model that is more "messy" with a large number of features.

> Features were selected using the sklean "Recursive Feature Elimination with Cross Validation" function, or RFECV. RFECV uses cross-validation and begins the model with all features, then eliminates the weakest feature and scores the model again, using cv with each iteration. At the end it returns the model with the feature set that produced the highest score on the cv. In the case of price predictions I used mean absolute error as my preferred scoring metric. Then, starting with the features recommended by RFECV, I ran the model again through a forward-backward feature selector, which starts with zero features and iteratively adds and subtracts features until all features have a p-value under the threshold (in this case .05). 

> Almost all included features had p-values so low that the probability that these features are influencing the price by random chance is near zero. Our highest included p-value, zipcode 98003, has only a .3% chance of having influenced prices by random chance.

> 88.2% of our variation in price can be explained by our model variables. Our equal R^2 and adjusted R^2 tells us that all of our features are contributing to our model. Adjusted R^2 penalizes the R^2 formula based on number of variables, so a same score tells us that our variables are all contributors.

> F-statistic and Prob(F-statistic) tell us if our variable group as a whole is statistically significant.The null hypothesis is a model with no variables (only intercept). Our prob(f-statistic) evaluates the chance that our null hypothesis is true and our variables have no effect. On our model this chance is so low that it's functionally 0. F-statistic is evaluating all of the coefficients used together, so p-value of individual coefficients is still important. So this number tells us that our MODEL is significant without telling us if any individual feature is significant.

> Omnibus and prob(omnibus) are telling us that our residuals are not normally distributed. We want to see a value close to 0 for Omnibus, and a value close to 1 for Prob(Omnibus). Neither of these things is the case. We saw heteroscedasticity on the residuals for all of our model types that we tried. All of our models seem to perform well at lower values and then variance increases as price increases. This was true on both linear and nonlinear approaches such as decision trees.

> Our data is skewed slightly left. Our data has high kurtosis which implies tighter grouping of our residuals around zero and fewer outliers, which implies a fitted model.

> Durbin Watson is a measure of homoscedasticity - an even distribution of errors. We are showing slight heteroscedasticity because we'd like to see this number between 1 and 2. We saw that heteroscedasticity in the residuals.

> Our low condition number implies no multicollinearity with our features.

> Overall our characteristics are mediocre. Our high omnibus and above 2 Durbin-Watson are problematic. But no better model was found to correct for the heteroscedasticity.

# Conclusions and Recommendations

#### What are the primary factors influencing housing prices in the King County metro area?
> As square footage increases so does quality of materials. Most importantly you can see the upward price trend with both increased square footage and materials grade. I was intrigued that our lower bound of data points is very linear, but as our square footage increases, the upper bound gradually breaks away with higher variance. 

>I ranked the 70 zip codes in King County by median home value, and used those ranks to color our data points.  Our low median zip codes have a low price per square footage, and price per square foot increases with zip code median, which makes sense, but also shows the importance of zip code to pricing. I found it interesting that while most zip codes exhibit a clear trend of price per square foot decreasing with increased total square footage, which is entirely normal, certain very high value zip codes seem to retain their high price per square foot regardless of total square footage. Certain zip codes seem immune to the usual price per square foot decay. 

> As they say, location is everything, and it is the primary influencing factor for a home price in the King County metro area. Our darkest areas, and therefore highest value sales, are clustered in and around Seattle to the west of Lake Washington and into the eastern lake cities of Bellevue and Redmond which are the technical employer hubs of the region. As we move away from Seattle and the tech hubs into the suburbs, our prices clearly go down.

> These three features alone explain 85% of the price variance.

#### Can we effectively use a regression model based system for realtors to determine a proper list price?
> Our regression model, while explaining over 88% of the price variance with our features, was nonetheless far from accurate in absolute terms. A mean average error of 55k in either direction is a huge variance to a home price - one that is so large that it renders the prediction much less meaningful and useful. Other models need to be explored, better data needs to be sourced, or easy-to-use features that an average realtor is capable of evaluating/acquiring should be added to the model to improve its predictive quality. The model is providing at best a baseline starting point.

#### Is a model-based system more accurate for determining list price than the traditional comps-based system?
>At present, the regression model is only slightly more accurate than the comps-based system on our test data - and this is even with our comps-based simulator lacking a feature set as robust as our model. In fact our realtor simulator performed better on new data than our model. The model is lacking something - quality start data, predictive features, proper feature engineering, or something else - and we should identify and include them.

> With our model one major pro is it is a mathematical model and we can use it without housing data on hand. It would be benefited by up to date data, but it doesn't require it to function. This certainly results in less labor overhead as it only needs occasional updating. However the wide error range gives only a starting point, and it ultimately requires human intervention to determine final price, which is likely a comps-based manual pass anyway.

> The comps based system, on the other hand, is a familiar method for realtors. It somehow more accurately captures the location granularity that the model misses. And most importantly, it can still be accomplished programmatically, as I've demonstrated, which can provide a quicker starting point for a realtor instead of them identifying the comps manually. The downside here is cost of maintenance - the comps finder requires the entire data set to be present to function, and it must be updated frequently, preferably daily, for most accurate results. With that in mind the realtor simulator requires a lot more overhead to remain usable.

#### What easy-to-use features can we add to our model to increase its accuracy?

>I would consider two different models that separate urban and suburban areas, to better capture the non-decaying price per square foot in our high value zip codes that we saw back on our square footage visualization.

> I'd source better data for this project. Data that includes the type of sale would be incredibly important to weed out non-market sales which may be hurting our model.  When I searched out new data on which to test our completed model I saw a large proportion of non-standard sales on record, and this adds noise to our data.

> I want to use latitude and longitude as a more granular location identifier in place of zip code

> If we add features to the model, they need to be ones that an average realtor is capable of finding and utilizing with average Google skills. We could add frontend GUI features and backend web scraping that allows some of these things to be run by the model software. For example, using latititude and longitude, GreatSchools.org can tell a property's exact school assignments and ratings. A realtor doesn't know lat and long, but we can use under the hood web scraping that takes the address and uses a reverse address finder such as at https://www.latlong.net/Show-Latitude-Longitude.html to determine lat/long, then takes that result to GreatSchools.org to scrape for school information. Lat/long can also be utilized within the model in combination with the many GIS map tools available from King County about public services, parks, school districts, area income and more to obtain and enter data without the realtor needing to do any more than enter an address.


# Visualizations

## Feature Visualizations

In [ ]:
# refresh on our original data frame
df

In [ ]:
# get the columns we are going to make visualizations from
viz_df = df[['price', 'sqft_living', 'median_zip', 'zip_rank', 'grade']]
viz_df['pr_sf'] = round(viz_df['price']/viz_df['sqft_living'], 2)
viz_df

In [ ]:
# make simpler variables for our visualiation variables
viz_target = viz_df['price']/100000
viz_sqft = viz_df['sqft_living']
viz_grade = viz_df['grade']
viz_zip = viz_df['zip_rank'] 
viz_zip2 = viz_df['median_zip']
viz_pr_sf = viz_df['pr_sf']

In [ ]:
fig, ax=plt.subplots(figsize=(20,15)) # prepare our figure

sns.set(font_scale = 1.5) # set our font scale bigger for this vis

# scatter our data
scatter2 = sns.scatterplot(x="sqft_living", y=viz_target, data=viz_df, hue='grade', palette='magma_r')

# label axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price in $100,000', fontsize=16)
ax.set_title("Price to Total Square Footage\nby Grade of Materials", fontsize=20)

# label and position our legend
plt.legend(title='Grade', loc='upper left', title_fontsize=20);

# save visualization to png
#plt.savefig('images/pr_grade.png')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))

#ax.scatter(viz_sqft, viz_pr_sf, c=viz_zip, cmap='magma_r')
scatter2 = sns.scatterplot(x="sqft_living", y="pr_sf", data=viz_df, hue='grade', palette='magma_r')

# label axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price per Square Foot', fontsize=16)
ax.set_title("Price per Square Foot to Total Square Footage\nby Grade of Materials", fontsize=20)

# label and position our legend
plt.legend(title='Grade', loc='upper right', title_fontsize=20);

# save visualization to png
#plt.savefig('images/pr_sf_grade.png');

In [ ]:
# prepare figure
fig, ax = plt.subplots(figsize=(20, 15))

#scatter our data
scatter3 = sns.scatterplot(x="sqft_living", y="price", data=viz_df, hue='zip_rank', palette='magma_r')
#ax.scatter(viz_sqft, viz_target, c=viz_zip, cmap='magma_r')

# label our axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price in $100,000', fontsize=16)
ax.set_title("Price per Total Square Footage\nby Zip Code Median Value Rank", fontsize=20);

# save visualization to png
#plt.savefig('images/sqft.png');

In [ ]:
# prepare figure
fig, ax = plt.subplots(figsize=(20, 15))

#scatter our data
scatter3 = sns.scatterplot(x="sqft_living", y="pr_sf", data=viz_df, hue='zip_rank', palette='magma_r')
#ax.scatter(viz_sqft, viz_target, c=viz_zip, cmap='magma_r')

# label our axes and title
ax.set_xlabel('Total Square Footage', fontsize=16)
ax.set_ylabel('Price per Square Foot', fontsize=16)
ax.set_title("Price per per Square Foot to Total Square Footage\nby Zip Code Median Value Rank", fontsize=20);

# save visualization to png
#plt.savefig('images/pr_sf_zip.png');

In [ ]:
viz_y = viz_df['price']
viz_x = viz_df.drop('price', axis=1)

fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(viz_sqft, viz_grade, viz_target, c=viz_zip, cmap='magma_r')
#ax.scatter(viz_sqft, viz_grade, viz_target, c='red', label="Predictions")
#ax.scatter(viz_sqft, viz_grade, end_z/100000, c='green', label="Actuals")

ax.set_xlabel('Square Feet of Living Space', fontsize=12)
ax.set_ylabel('Grade of Materials', fontsize=12)
ax.set_zlabel('Price', fontsize=12)

ax.set_title("Price per Square Footage and Grade of Materials, by Zip Median Rank", fontsize=20)

# first num is tilt angle, second num is turn angle
# default is about 30,305
# 0, 270 creates side view of pr/sqft
# 0, 360 creates side view of pr/grade
ax.view_init(30, 305)


# save visualization to png
#plt.savefig('images/3d_feats.png');

# DEPRECATED

In [22]:
 load and look at our austin housing data
df = pd.read_csv('newyork_housing.csv')
df.head()

address/city  address/community address/neighborhood address/state  \
0     New York                NaN                  NaN            NY   
1        Bronx                NaN                  NaN            NY   
2        Bronx                NaN                  NaN            NY   
3        Bronx                NaN                  NaN            NY   
4        Bronx                NaN                  NaN            NY   

  address/streetAddress address/subdivision  address/zipcode  bathrooms  \
0   60 Terrace View Ave                 NaN          10463.0        2.0   
1        625 W 246th St                 NaN          10471.0        8.0   
2        716 W 231st St                 NaN          10463.0        3.0   
3        750 W 232nd St                 NaN          10463.0        6.0   
4        632 W 230th St                 NaN          10463.0        6.0   

   bedrooms currency    dateposted  \
0       5.0      USD  1.610134e+12   
1       8.0      USD  1.595968e+12   
2       4.0      USD  1.592668e+12   
3       5.0      USD  1.600814e+12   
4       5.0      USD  1.605751e+12   

                                         description homeStatus   latitude  \
0  Discover Marble Hill, a neighborhood rich with...   FOR_SALE  40.877743   
1  EXCLUSIVE BRAND NEW\nLavish Newly Built 8-Bd. ...   FOR_SALE  40.892689   
2  This 4233 square foot single family home has 4...   FOR_SALE  40.883419   
3  EXCLUSIVE NEW TO MARKET\nPrime Renovation Oppo...   FOR_SALE  40.885033   
4  EXCLUSIVE JUST LISTED\nNewly Built 5-Bd. Stucc...   FOR_SALE  40.881702   

   livingArea  longitude                                           photos/0  \
0      1889.0 -73.910866  https://photos.zillowstatic.com/fp/006eb7ab9c3...   
1      7000.0 -73.910667  https://photos.zillowstatic.com/fp/9c91425f8fb...   
2      4233.0 -73.918106  https://photos.zillowstatic.com/fp/797eb4a9695...   
3      7000.0 -73.917793  https://photos.zillowstatic.com/fp/36a83a8d300...   
4         NaN -73.914185  https://photos.zillowstatic.com/fp/daa98ebe0c5...   

                                            photos/1  \
0  https://photos.zillowstatic.com/fp/8b10648864c...   
1  https://photos.zillowstatic.com/fp/82a019ae300...   
2  https://photos.zillowstatic.com/fp/956034ac704...   
3  https://photos.zillowstatic.com/fp/90000a23b0b...   
4  https://photos.zillowstatic.com/fp/fe816fa637b...   

                                            photos/2  \
0  https://photos.zillowstatic.com/fp/08edf887608...   
1  https://photos.zillowstatic.com/fp/d1d0ed56f85...   
2  https://photos.zillowstatic.com/fp/b799eae9027...   
3  https://photos.zillowstatic.com/fp/9d7bb173624...   
4  https://photos.zillowstatic.com/fp/2b2e0075694...   

                                            photos/3  \
0  https://photos.zillowstatic.com/fp/d61f7b6017e...   
1  https://photos.zillowstatic.com/fp/65103165fc7...   
2  https://photos.zillowstatic.com/fp/4b393c3aff3...   
3  https://photos.zillowstatic.com/fp/6e171567e67...   
4  https://photos.zillowstatic.com/fp/ccfe5f7baf0...   

                                            photos/4  \
0  https://photos.zillowstatic.com/fp/4a8d68dccb2...   
1  https://photos.zillowstatic.com/fp/8d20b0ec4ef...   
2  https://photos.zillowstatic.com/fp/a207d715629...   
3  https://photos.zillowstatic.com/fp/f75d80b80be...   
4  https://photos.zillowstatic.com/fp/f2c81a6d161...   

                                            photos/5  \
0  https://photos.zillowstatic.com/fp/9fd2c273f39...   
1  https://photos.zillowstatic.com/fp/7b9f904cb69...   
2  https://photos.zillowstatic.com/fp/5724a2d40d9...   
3  https://photos.zillowstatic.com/fp/8c2d97e9c45...   
4  https://photos.zillowstatic.com/fp/d0f78479941...   

                                            photos/6  \
0  https://photos.zillowstatic.com/fp/7bc3ee9bc4c...   
1  https://photos.zillowstatic.com/fp/20e8df506bb...   
2  https://photos.zillowstatic.com/fp/e4f91d8b114...   
3  https://photos.zillowstatic.com/f

In [23]:
df.shape

(75630, 1507)

In [24]:
df.drop(df.loc[:,df.columns.str.contains('factLabel')], axis=1, inplace=True)

In [25]:
df.drop(df.loc[:,df.columns.str.contains('photos')], axis=1, inplace=True)

In [26]:
df.isna().sum().head(20)

address/city                                      1
address/community                             75630
address/neighborhood                          74953
address/state                                     1
address/streetAddress                             1
address/subdivision                           75628
address/zipcode                                  19
bathrooms                                     19053
bedrooms                                      19464
currency                                          0
dateposted                                    52200
description                                    6684
homeStatus                                        0
latitude                                         26
livingArea                                     9211
longitude                                        26
price                                            39
priceHistory                                  75630
priceHistory/0/attributeSource/infoString1    51477
priceHistory

In [27]:
df.drop(df.loc[df['bathrooms'].isna()].index, axis=0, inplace=True)
df.drop(df.loc[df['bedrooms'].isna()].index, axis=0, inplace=True)
df.drop(df.loc[df['livingArea'].isna()].index, axis=0, inplace=True)

In [32]:
df.isna().sum().head(20)

address/city                                      0
address/community                             49943
address/neighborhood                          49315
address/state                                     0
address/streetAddress                             0
address/subdivision                           49941
address/zipcode                                   1
bathrooms                                         0
bedrooms                                          0
currency                                          0
dateposted                                    32138
description                                    4087
homeStatus                                        0
latitude                                         23
livingArea                                        0
longitude                                        23
price                                            29
priceHistory                                  49943
priceHistory/0/attributeSource/infoString1    31267
priceHistory

In [33]:
limit = len(df) * .25
#df.dropna(thresh=limit, axis=1, inplace=True)

In [34]:
df.isna().sum().head(20)

address/city                                      0
address/state                                     0
address/streetAddress                             0
address/zipcode                                   1
bathrooms                                         0
bedrooms                                          0
currency                                          0
dateposted                                    32138
description                                    4087
homeStatus                                        0
latitude                                         23
livingArea                                        0
longitude                                        23
price                                            29
priceHistory/0/attributeSource/infoString1    31267
priceHistory/0/attributeSource/infoString2     3534
priceHistory/0/buyerAgent/name                34172
priceHistory/0/buyerAgent/photo/url           34831
priceHistory/0/buyerAgent/profileUrl          34172
priceHistory

In [37]:
df.columns

Index(['address/city', 'address/state', 'address/streetAddress',
       'address/zipcode', 'bathrooms', 'bedrooms', 'currency', 'dateposted',
       'description', 'homeStatus',
       ...
       'schools/2/link', 'schools/2/name', 'schools/2/rating',
       'schools/2/size', 'schools/2/studentsPerTeacher',
       'schools/2/totalCount', 'schools/2/type', 'url', 'yearBuilt', 'zpid'],
      dtype='object', length=210)

In [38]:
df['homeStatus'].unique()

array(['FOR_SALE', 'SOLD', 'RECENTLY_SOLD', 'PRE_FORECLOSURE', 'FOR_RENT',
       'FORECLOSED'], dtype=object)

In [39]:
df.drop(df.loc[df['homeStatus'] == 'FOR_RENT'].index, axis=0, inplace=True)
df.drop(df.loc[df['homeStatus'] == 'PRE_FORECLOSURE'].index, axis=0, inplace=True)
df.drop(df.loc[df['homeStatus'] == 'FORECLOSED'].index, axis=0, inplace=True)
df.drop(df.loc[df['homeStatus'] == 'OTHER'].index, axis=0, inplace=True)

In [40]:
df.loc[df['priceHistory/0/postingIsRental'] == True]

,address/city,address/state,address/streetAddress,address/zipcode,bathrooms,bedrooms,currency,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/attributeSource/infoString1,priceHistory/0/attributeSource/infoString2,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/showCountyLink,priceHistory/0/source,priceHistory/0/time,priceHistory/1/attributeSource/infoString1,priceHistory/1/attributeSource/infoString2,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/source,priceHistory/1/time,priceHistory/2/attributeSource/infoString1,priceHistory/2/attributeSource/infoString2,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/source,priceHistory/2/time,priceHistory/3/attributeSource/infoString1,priceHistory/3/attributeSource/infoString2,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/source,priceHistory/3/time,priceHistory/4/attributeSource/infoString1,priceHistory/4/attributeSource/infoString2,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/source,priceHistory/4/time,priceHistory/5/attributeSource/infoString1,priceHistory/5/attributeSource/infoString2,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/source,priceHistory/5/time,priceHistory/6/attributeSource/infoString2,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/source,priceHistory/6/time,priceHistory/7/attributeSource/infoString2,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/source,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/0/factValue,resoFactsStats/atAGlanceFacts/1/factValue,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/canRaiseHorses,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAdditionalParcels,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasHomeWarranty,resoFactsStats/hasLandLease,resoFactsStats/hasOpenParking,resoFactsStats/hasPetsAllowed,resoFactsStats/hasPrivatePool,resoFactsStats/hasRentControl,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,

In [41]:
df.drop(df.loc[df['priceHistory/0/postingIsRental'] == True].index, axis=0, inplace=True)

In [42]:
df.replace({False: 0, True: 1}, inplace=True)

In [43]:
df.describe()

,address/zipcode,bathrooms,bedrooms,dateposted,latitude,livingArea,longitude,price,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/showCountyLink,priceHistory/0/time,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/canRaiseHorses,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAdditionalParcels,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasHomeWarranty,resoFactsStats/hasLandLease,resoFactsStats/hasOpenParking,resoFactsStats/hasPetsAllowed,resoFactsStats/hasPrivatePool,resoFactsStats/hasRentControl,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/isNewConstruction,resoFactsStats/onMarketDate,resoFactsStats/parking,resoFactsStats/stories,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuilt,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/isAssigned,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/1/distance,schools/1/isAssigned,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/totalCount,schools/2/distance,schools/2/isAssigned,schools/2/rating,schools/2/size,schools/2/studentsPerTeacher,schools/2/totalCount,yearBuilt,zpid
count,46791.000000,46792.000000,46792.000000,1.720600e+04,46791.000000,46792.000000,46791.000000,4.676300e+04,46784.0,4.674700e+04,4.674900e+04,46784.0,4.678400e+04,41688.000000,4.159600e+04,4.160300e+04,41688.0,4.168800e+04,39280.000000,3.918100e+04,3.918500e+04,39280.0,3.928000e+04,33483.000000,3.335100e+04,33365.000000,33483.0,3.348300e+04,27240.000000,2.706300e+04,2.708800e+04,27240.0,2.724000e+04,21916.000000,2.171100e+04,2.174200e+04,21916.0,2.191600e+04,17552.000000,1.734500e+04,1.736600e+04,17552.0,1.755200e+04,13933.000000,1.375400e+04,1.377000e+04,13933.0,1.393300e+04,46785.000000,45955.000000,44399.000000,44382.000000,31528.000000,39119.000000,46792.000000,46792.0,46792.000000,46792.000000,46792.0,46792.000000,46792.000000,46792.000000,45998.000000,14680.000000,46792.000000,46615.000000,46792.0,46792.0,46792.00000,46792.000000,18131.000000,46792.0,46792.000000,46792.000000,13628.000000,1.720600e+04,46792.000000,29817.000000,4.102400e+04,4.023600e+04,44439.000000,2.456400e+04,46722.000000,46722.0,46203.000000,46444.000000,46184.000000,46722.000000,46268.000000,46268.0,46192.000000,46173.000000,45315.000000,46268.000000,35167.000000,35167.0,35147.000000,35139.000000,34831.000000,35167.000000,44419.000000,4.679200e+04
mean,10838.975551,2.822816,4.042400,1.603914e+12,40.693087,2775.702919,-73.944114,1.003919e+06,0.0,1.009257e+06,2.803336e+02,0.0,1.570025e+12,0.037853,9.601810e+05,2.313589e+02,0.0,1.530793e+12,0.040020,9.366302e+05,5.960665e+02,0.0,1.521413e+12,0.039871,9.003

In [44]:
df['address/city'].unique()

array(['New York', 'Bronx', 'Manhattan', 'new york', 'New york', 'Street',
       'Pelham', 'Howard Beach', 'Broad Channel', 'Far Rockaway',
       'Jamaica', 'Rosedale', 'Rockaway Park', 'Belle Harbor', 'Neponsit',
       'Breezy Point', 'Queens', 'Far rockaway', 'BELLE HARBOR',
       'Belle harbor', 'rosedale', 'Rockaway point', 'Far Rockway',
       'Washington Heights', 'Avenue', 'Brooklyn', 'Maspeth',
       'Little Neck', 'Flushing', 'NEW YORK', 'Staten Island',
       'Staten island', 'staten Island', 'staten island', 'BROOKLYN',
       'Cambria Heights', 'Queens Village', 'Springfield Gardens',
       'Arverne', 'far rockaway', 'Lawrence', 'belle harbor',
       'New york City', 'NY', 'Yonkers', 'College', 'bronx', 'Cen',
       'elmont', 'boulevard', 'Breezy Pt', 'Rockaway Beach',
       'Rockaway Point', 'Rockaway park', '350w42ndst', 'Oval', 'BRONX',
       'East 214th Street', 'West 156th', 'Concourse', 'Plains',
       'New York CIty', 'New York City', 'West', 'Long Islan

In [45]:
df['resoFactsStats/cityRegion'].unique()

array(['New York', 'Bronx', 'Manhattan', 'Street', 'Pelham',
       'Howard Beach', 'Broad Channel', 'Far Rockaway', 'Jamaica',
       'Rosedale', 'Rockaway Park', 'Belle Harbor', 'Neponsit',
       'Breezy Point', 'Queens', 'Rockaway Point', 'Far Rockway',
       'Washington Heights', 'Avenue', 'Brooklyn', 'Maspeth',
       'Little Neck', 'Flushing', 'Staten Island', 'Cambria Heights',
       'Queens Village', 'Springfield Gardens', 'Arverne', 'Lawrence',
       'New York City', 'Ny', 'Yonkers', 'College', 'Cen', 'Elmont',
       'Boulevard', 'Breezy Pt', 'Rockaway Beach', '350 W 42 Ndst',
       'Oval', 'East 214th Street', 'West 156th', 'Concourse', 'Plains',
       'West', 'Long Island City', 'Astoria', 'College Pt',
       'East Elmhurst', 'Woodside', 'College Point', 'Corona',
       'Sunnyside', 'Ridgewood', 'Greenpoint', 'Bayside', 'Whitestone',
       'Beechhurst', '16th', 'Douglaston', 'Douglas Manor', 'Floral Park',
       'Great Neck', 'Flushing Ny 11355 Queensboro Hills',


In [46]:
df.drop(df.loc[:,df.columns.str.contains('source')], axis=1, inplace=True)
df.drop(df.loc[:,df.columns.str.contains('Source')], axis=1, inplace=True)
df.drop(['currency', 'resoFactsStats/atAGlanceFacts/1/factValue', 'resoFactsStats/atAGlanceFacts/0/factValue', 'priceHistory/0/postingIsRental', 
        'priceHistory/0/showCountyLink', 'resoFactsStats/canRaiseHorses', 'resoFactsStats/yearBuilt', 'zpid',
        'url', 'schools/0/link', 'schools/1/link', 'resoFactsStats/hasAdditionalParcels', 'resoFactsStats/hasHomeWarranty', 
        'resoFactsStats/hasLandLease', 'resoFactsStats/hasRentControl', 'schools/0/isAssigned', 'schools/1/isAssigned',
        'resoFactsStats/hasPetsAllowed', 'schools/0/grades', 'schools/1/grades', 'address/state',], axis=1, inplace=True)

In [47]:
df

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [48]:
df['resoFactsStats/homeType'].unique()

array(['Residential', 'Single Family', 'Condo', 'Apartment',
       'Multiple Occupancy', 'Townhouse', 'Residential Income', 'Other',
       'Mobile / Manufactured', 'Vacant Land'], dtype=object)

In [49]:
df.drop(df[(df['resoFactsStats/homeType'] == 'Multiple Occupancy') | (df['resoFactsStats/homeType'] == 'Vacant Land') | 
           (df['resoFactsStats/homeType'] == 'Residential Income') | (df['resoFactsStats/homeType'] == 'Land') |
           (df['resoFactsStats/homeType'] == 'Other') | (df['resoFactsStats/homeType'] == 'Mixed Use') |
            (df['resoFactsStats/homeType'] == 'Mobile / Manufactured')].index, inplace=True)

In [50]:
df['resoFactsStats/homeType'].unique()

array(['Residential', 'Single Family', 'Condo', 'Apartment', 'Townhouse'],
      dtype=object)

In [51]:
df['schools/0/type'].unique()

array(['Public', nan, 'Charter'], dtype=object)

In [52]:
df['schools/0/totalCount'].unique()

array([ 1.,  3.,  2., nan,  7.,  4.,  6.,  8.,  5., 16., 11., 17.])

In [53]:
df.loc[df['resoFactsStats/garageSpaces'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [54]:
df.loc[df['resoFactsStats/furnished'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [55]:
df.loc[df['resoFactsStats/hasCarport'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [56]:
df.loc[df['resoFactsStats/hasOpenParking'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [57]:
df.loc[df['resoFactsStats/hasSpa'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [58]:
df.loc[df['resoFactsStats/hasAttachedProperty'] > 0]

,address/city,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/furnished,resoFactsStats/garageSpaces,resoFactsStats/hasAttachedGarage,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasSpa,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/0/type,schools/1/distance,scho

In [59]:
df.drop([ 'address/city', 'schools/0/type', 'schools/1/type', 'resoFactsStats/garageSpaces', 'resoFactsStats/hasAttachedGarage', 'resoFactsStats/furnished', 'resoFactsStats/hasSpa'], axis=1, inplace=True)

In [60]:
df.describe()

,address/zipcode,bathrooms,bedrooms,dateposted,latitude,livingArea,longitude,price,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/time,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasView,resoFactsStats/isNewConstruction,resoFactsStats/onMarketDate,resoFactsStats/parking,resoFactsStats/stories,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/1/distance,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/totalCount,schools/2/distance,schools/2/isAssigned,schools/2/rating,schools/2/size,schools/2/studentsPerTeacher,schools/2/totalCount,yearBuilt
count,33843.000000,33844.000000,33844.000000,1.322600e+04,33843.000000,33844.000000,33843.000000,3.383500e+04,3.382500e+04,3.382600e+04,3.383800e+04,30031.000000,2.997500e+04,2.998100e+04,30031.0,3.003100e+04,28315.000000,2.823600e+04,2.824000e+04,28315.0,2.831500e+04,24125.000000,2.403000e+04,24040.000000,24125.0,2.412500e+04,19526.000000,1.940000e+04,19417.000000,19526.0,1.952600e+04,15627.000000,1.546600e+04,1.549300e+04,15627.0,1.562700e+04,12452.000000,1.231300e+04,1.232500e+04,12452.0,1.245200e+04,9922.000000,9.793000e+03,9805.000000,9922.0,9.922000e+03,33841.000000,33172.000000,32312.000000,32306.000000,21811.000000,29412.000000,33844.000000,33844.000000,33844.000000,33306.000000,10692.000000,33844.000000,33721.000000,33844.000000,14605.000000,33844.000000,11180.000000,1.322600e+04,33844.000000,22646.000000,2.929500e+04,2.863300e+04,1.688900e+04,33792.000000,33505.000000,33679.000000,33529.000000,33792.000000,33517.000000,33484.000000,33477.000000,32885.000000,33517.000000,26316.000000,26316.0,26310.000000,26308.000000,26058.000000,26316.000000,31970.000000
mean,10794.807553,2.602683,3.435941,1.603643e+12,40.678698,2730.136509,-73.960902,9.667339e+05,9.698192e+05,1.919423e+02,1.570375e+12,0.035297,9.306185e+05,1.327336e+02,0.0,1.533056e+12,0.034469,9.154616e+05,4.778968e+02,0.0,1.523948e+12,0.033326,9.014649e+05,288.731967,0.0,1.464040e+12,0.042661,9.179562e+05,177.477669,0.0,1.434414e+12,0.042875,8.344054e+05,2.307985e+02,0.0,1.415150e+12,0.042644,8.180058e+05,3.192425e+02,0.0,1.397651e+12,0.043540,8.127664e+05,247.243055,0.0,1.381606e+12,0.836489,2.610756,1.751145,0.588374,0.003072,0.187916,3.435941,0.099013,0.047571,0.502282,0.468294,0.273579,0.565375,0.069732,0.190962,0.091892,0.067800,1.596825e+12,0.525056,2.595469,1.833464e+04,1.175702e+06,3.143208e+03,0.409884,6.387733,713.287033,13.900355,1.177734,0.944407,5.766396,1047.364160,14.345659,1.252827,1.330974,1.0,3.855720,2279.162802,17.61

In [61]:
df.reset_index(inplace=True, drop=True)

In [62]:
df

,address/streetAddress,address/zipcode,bathrooms,bedrooms,dateposted,description,homeStatus,latitude,livingArea,longitude,price,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/event,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/time,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/showCountyLink,priceHistory/1/time,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/showCountyLink,priceHistory/2/time,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/showCountyLink,priceHistory/3/time,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/showCountyLink,priceHistory/4/time,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/showCountyLink,priceHistory/5/time,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/showCountyLink,priceHistory/6/time,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/showCountyLink,priceHistory/7/time,propertyTaxRate,resoFactsStats/appliances/0,resoFactsStats/appliances/1,resoFactsStats/appliances/2,resoFactsStats/atAGlanceFacts/2/factValue,resoFactsStats/atAGlanceFacts/3/factValue,resoFactsStats/atAGlanceFacts/4/factValue,resoFactsStats/atAGlanceFacts/5/factValue,resoFactsStats/basement,resoFactsStats/bathrooms,resoFactsStats/bathroomsFull,resoFactsStats/bathroomsHalf,resoFactsStats/bathroomsOneQuarter,resoFactsStats/bathroomsThreeQuarter,resoFactsStats/bedrooms,resoFactsStats/cityRegion,resoFactsStats/constructionMaterials/0,resoFactsStats/cooling/0,resoFactsStats/elementarySchoolDistrict,resoFactsStats/flooring/0,resoFactsStats/hasAttachedProperty,resoFactsStats/hasCarport,resoFactsStats/hasCooling,resoFactsStats/hasFireplace,resoFactsStats/hasGarage,resoFactsStats/hasHeating,resoFactsStats/hasOpenParking,resoFactsStats/hasPrivatePool,resoFactsStats/hasView,resoFactsStats/heating/0,resoFactsStats/heating/1,resoFactsStats/highSchoolDistrict,resoFactsStats/homeType,resoFactsStats/isNewConstruction,resoFactsStats/livingArea,resoFactsStats/lotSize,resoFactsStats/middleOrJuniorSchoolDistrict,resoFactsStats/onMarketDate,resoFactsStats/otherFacts/0/name,resoFactsStats/otherFacts/0/value,resoFactsStats/otherFacts/1/name,resoFactsStats/otherFacts/1/value,resoFactsStats/otherFacts/2/name,resoFactsStats/otherFacts/2/value,resoFactsStats/otherFacts/3/name,resoFactsStats/otherFacts/3/value,resoFactsStats/otherFacts/4/name,resoFactsStats/otherFacts/4/value,resoFactsStats/otherFacts/5/name,resoFactsStats/otherFacts/5/value,resoFactsStats/otherFacts/6/name,resoFactsStats/otherFacts/6/value,resoFactsStats/otherFacts/7/name,resoFactsStats/otherFacts/7/value,resoFactsStats/otherFacts/8/name,resoFactsStats/otherFacts/8/value,resoFactsStats/otherFacts/9/name,resoFactsStats/otherFacts/9/value,resoFactsStats/otherFacts/10/value,resoFactsStats/otherFacts/11/value,resoFactsStats/parcelNumber,resoFactsStats/parking,resoFactsStats/parkingFeatures/0,resoFactsStats/parkingFeatures/1,resoFactsStats/rooms/0/roomType,resoFactsStats/sewer/0,resoFactsStats/stories,resoFactsStats/structureType,resoFactsStats/taxAnnualAmount,resoFactsStats/taxAssessedValue,resoFactsStats/yearBuiltEffective,schools/0/distance,schools/0/level,schools/0/name,schools/0/rating,schools/0/size,schools/0/studentsPerTeacher,schools/0/totalCount,schools/1/distance,schools/1/level,schools/1/name,schools/1/rating,schools/1/size,schools/1/studentsPerTeacher,schools/1/totalCount,schools/2/distance,schools/

In [ ]:
df.to_pickle("listings_cleaned_na.pkl")